In [1]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 
import json 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)



In [2]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib_doubles(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_1 = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_1 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    # Now we need to match RA1 and Dec1 of t1 to RA2 and Dec2 of t2 in case the pairs are mixeup between tables
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA2_deg']*u.degree, dec=t2['Dec2_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1_2, idx2_2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1_2)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2_2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_2 = pd.concat([(t1.iloc[idx1_2]),(t2.iloc[idx2_2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_2 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    return tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2





In [3]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib_singles(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
    return tunique, tmatches, idx1, idx2





In [4]:
# load in individual table



# load in double-peaked table
dp = pd.read_csv('Double-peaked_dual_agn_candidates.csv', sep=',')
dp = dp.fillna('-99')
for index, row in dp.iterrows():
    if row['RA1']==row['RA2']:
        dp.at[index, 'RA2']='-99'
        dp.at[index, 'Dec2']='-99'
dp.reset_index(drop=True, inplace=True)

# load in binary quasar table
bqs = pd.read_csv('Binary_quasars_tables_DR1.csv', sep=',')
# reminder to go back and add in naming conventions for the tytler2009 pairs...
bqs = bqs.fillna('-99')
bqs.reset_index(drop=True, inplace=True)

# load in lens table
lens = pd.read_csv('Duals_from_lens_searches.csv', sep=',')
#lens['Selection Method'] = '-99'
lens = lens.fillna('-99')
lens.reset_index(drop=True, inplace=True)

# load in x-shaped radio source
xshape = pd.read_csv('Xshaped_radio_samples_DR1.csv', sep=',')
# reminder to add in the Wang2003 paper that Lal2019 talks about but is not in the Cheung+ table
xshape = xshape.fillna('-99')
xshape.reset_index(drop=True, inplace=True)

# load in periodicity table
periodicity = pd.read_csv('Periodicity_catalogs_DR1.csv', sep=',')
periodicity = periodicity.fillna('-99')
periodicity.reset_index(drop=True, inplace=True)

# load in binary candidate table
binary_BL = pd.read_csv('Binary_broadline_tables_DR1.csv', sep=',')
binary_BL = binary_BL.fillna('-99')
binary_BL.reset_index(drop=True, inplace=True)

# load in offset AGN/recoil table
offset = pd.read_csv('Offset_AGN_tables_DR1.csv', sep=',')
# reminder that we need to go back and add in missing tables from Comerford's work and Barrows' work
offset = offset.fillna('-99')
offset.reset_index(drop=True, inplace=True)

# load varstrometry tables
varstr = pd.read_csv('Varstrometry_tables_DR1.csv', sep=',')
varstr = varstr.fillna('-99')
varstr.reset_index(drop=True, inplace=True)

# load in optical spectroscopic line ratio catalog
specrat = pd.read_csv('Opticalspec_radiosel_DR1.csv', sep=',')
specrat = specrat.fillna('-99')
specrat.reset_index(drop=True, inplace=True)
# this has Name1



In [5]:
#bqs

In [6]:
#xshape

In [7]:
# adding in now the BPT selected AGNs and the ratio selected AGNs from Liu and Fu
# now we'll tackle matching the binary quasar tables against the MAC
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(dp,specrat,0.1)

print(len(idx1),len(idx1_2)) 
# we're matching with a small tolerance first and collect the relevant information from the double-peak tables, \
# remove those objects from the list, and then rematch at hgiher tolerance. This will ensure we do not match \
# multiples for the same merger in the spectroscopic ratio + radio table.

# FOR THIS CASE: IDX1 IS FOR DP AND IDX2 IS FOR SPECRAT
# USUALLY I AM MATCHING THE MAC AGAINST THE OTHER TABLE, BUT HERE SINCE IM DEFERRING TO LIU AND MATCHING AGAINST 
# LIU'S RA1 AND RA2, I'M REVERSING IT!

for i, j in zip(idx2, idx1):
    specrat.at[i, 'Selection Method'] += ' / ' + dp.at[j, 'Selection Method']
    specrat.at[i, 'Paper(s)'] += ' ; ' + dp.at[j, 'Paper(s)']
    specrat.at[i, 'BibCode(s)'] += ' ; ' + dp.at[j, 'BibCode(s)']
    specrat.at[i, 'DOI(s)'] += ' ; ' + dp.at[j, 'DOI(s)']
    specrat.at[i, 'System Type'] += ' / ' + dp.at[j, 'System Type']
    if specrat.at[i, 'Confirmation Method'] != dp.at[j, 'Confirmation Method']:
        specrat.at[i, 'Confirmation Method'] += ' / ' + dp.at[j, 'Confirmation Method']
    specrat.at[i, 'Notes'] += '. ' + dp.at[j, 'Notes']
    specrat.at[i, 'Notes'] += ' AGN 1 is a double-peaked AGN.'

for i, j in zip(idx2_2, idx1_2):
    specrat.at[i, 'Selection Method'] += ' / ' + dp.at[j, 'Selection Method']
    specrat.at[i, 'Paper(s)'] += ' ; ' + dp.at[j, 'Paper(s)']
    specrat.at[i, 'BibCode(s)'] += ' ; ' + dp.at[j, 'BibCode(s)']
    specrat.at[i, 'DOI(s)'] += ' ; ' + dp.at[j, 'DOI(s)']
    specrat.at[i, 'System Type'] += ' / ' + dp.at[j, 'System Type']
    if specrat.at[i, 'Confirmation Method'] != dp.at[j, 'Confirmation Method']:
        specrat.at[i, 'Confirmation Method'] += ' / ' + dp.at[j, 'Confirmation Method']
    specrat.at[i, 'Notes'] += '. ' + dp.at[j, 'Notes']
    specrat.at[i, 'Notes'] += ' AGN 2 is a double-peaked AGN.'

# dropping the matching indices from the dp table
dp.drop(idx1, axis=0, inplace=True)
dp.drop(idx1_2, axis=0, inplace=True)
dp.reset_index(drop=True, inplace=True)
#

# now repeating with a larger match tolerance OF 1''
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(dp,specrat,1)

print(len(idx1),len(idx1_2)) 

for i, j in zip(idx2, idx1):
    specrat.at[i, 'Selection Method'] += ' / ' + dp.at[j, 'Selection Method']
    specrat.at[i, 'Paper(s)'] += ' ; ' + dp.at[j, 'Paper(s)']
    specrat.at[i, 'BibCode(s)'] += ' ; ' + dp.at[j, 'BibCode(s)']
    specrat.at[i, 'DOI(s)'] += ' ; ' + dp.at[j, 'DOI(s)']
    specrat.at[i, 'System Type'] += ' / ' + dp.at[j, 'System Type']
    if specrat.at[i, 'Confirmation Method'] != dp.at[j, 'Confirmation Method']:
        specrat.at[i, 'Confirmation Method'] += ' / ' + dp.at[j, 'Confirmation Method']
    specrat.at[i, 'Notes'] += '. ' + dp.at[j, 'Notes']
    specrat.at[i, 'Notes'] += ' AGN 1 is a double-peaked AGN.'

for i, j in zip(idx2_2, idx1_2):
    specrat.at[i, 'Selection Method'] += ' / ' + dp.at[j, 'Selection Method']
    specrat.at[i, 'Paper(s)'] += ' ; ' + dp.at[j, 'Paper(s)']
    specrat.at[i, 'BibCode(s)'] += ' ; ' + dp.at[j, 'BibCode(s)']
    specrat.at[i, 'DOI(s)'] += ' ; ' + dp.at[j, 'DOI(s)']
    specrat.at[i, 'System Type'] += ' / ' + dp.at[j, 'System Type']
    if specrat.at[i, 'Confirmation Method'] != dp.at[j, 'Confirmation Method']:
        specrat.at[i, 'Confirmation Method'] += ' / ' + dp.at[j, 'Confirmation Method']
    specrat.at[i, 'Notes'] += '. ' + dp.at[j, 'Notes']
    specrat.at[i, 'Notes'] += ' AGN 2 is a double-peaked AGN.'

# dropping the matching indices from the dp table
dp.drop(idx1, axis=0, inplace=True)
dp.drop(idx1_2, axis=0, inplace=True)
dp.reset_index(drop=True, inplace=True)

# now repeating with a larger match tolerance of 30''
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(dp,specrat,30)

print(len(idx1),len(idx1_2))

for i, j in zip(idx2, idx1):
    specrat.at[i, 'Selection Method'] += ' / ' + dp.at[j, 'Selection Method']
    specrat.at[i, 'Paper(s)'] += ' ; ' + dp.at[j, 'Paper(s)']
    specrat.at[i, 'BibCode(s)'] += ' ; ' + dp.at[j, 'BibCode(s)']
    specrat.at[i, 'DOI(s)'] += ' ; ' + dp.at[j, 'DOI(s)']
    specrat.at[i, 'System Type'] += ' / ' + dp.at[j, 'System Type']
    if specrat.at[i, 'Confirmation Method'] != dp.at[j, 'Confirmation Method']:
        specrat.at[i, 'Confirmation Method'] += ' / ' + dp.at[j, 'Confirmation Method']
    specrat.at[i, 'Notes'] += '. ' + dp.at[j, 'Notes']
    specrat.at[i, 'Notes'] += ' AGN 1 is a double-peaked AGN.'

for i, j in zip(idx2_2, idx1_2):
    specrat.at[i, 'Selection Method'] += ' / ' + dp.at[j, 'Selection Method']
    specrat.at[i, 'Paper(s)'] += ' ; ' + dp.at[j, 'Paper(s)']
    specrat.at[i, 'BibCode(s)'] += ' ; ' + dp.at[j, 'BibCode(s)']
    specrat.at[i, 'DOI(s)'] += ' ; ' + dp.at[j, 'DOI(s)']
    specrat.at[i, 'System Type'] += ' / ' + dp.at[j, 'System Type']
    if specrat.at[i, 'Confirmation Method'] != dp.at[j, 'Confirmation Method']:
        specrat.at[i, 'Confirmation Method'] += ' / ' + dp.at[j, 'Confirmation Method']
    specrat.at[i, 'Notes'] += '. ' + dp.at[j, 'Notes']
    specrat.at[i, 'Notes'] += ' AGN 2 is a double-peaked AGN.'

# dropping the matching indices from the dp table
dp.drop(idx1, axis=0, inplace=True)
dp.drop(idx1_2, axis=0, inplace=True)
dp.reset_index(drop=True, inplace=True)


# now repeating with a larger match tolerance of 60''
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(dp,specrat,60)

print(len(idx1),len(idx1_2)) 
# Out to 60'' there are NO matches. So we should be good at this point.


# this process if verified


10 7
3 1
3 1
0 0


In [8]:
# now generating the MAC starting with the spectroscopic ratio + radio + double-peaked sources
MAC = pd.concat([specrat,dp])
MAC.reset_index(drop=True, inplace=True)


In [9]:
# now we'll tackle matching the binary quasar tables against the MAC
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(MAC,bqs,50)

print(len(tmatches_1),len(tmatches_2))
# No matches between MAC and the binary quasar list as of this point!

# verified this process

0 0


In [10]:
# Now concatenating the MAC with the binary quasar table!
MAC = pd.concat([MAC,bqs])
MAC.reset_index(drop=True, inplace=True)

In [11]:
# now adding in all of those dual AGN+candidates that turned up from lens searches

lens_nocoord = lens[lens['RA1']=='-99']
lens = lens[lens['RA1']!='-99']

# first dividing the MAC into a table with RA2 values and one without
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']


In [12]:
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(lens,MAC,0.1)
#tunique, tmatches, idx1, idx2 

print(len(idx1),len(idx1_2)) #17 and 17 matches to MAC's RA1 and RA2 for match tol of 0.1''

for i in idx1:
    if i in idx1_2:
        print('Stop, there is a duplicate match.')
        break
        

# FOR THIS CASE: IDX1 IS FOR lens AND IDX2 IS FOR MAC
# USUALLY I AM MATCHING THE MAC AGAINST THE OTHER TABLE, BUT HERE SINCE IM DEFERRING TO LIU AND MATCHING AGAINST 
# LIU'S RA1 AND RA2, I'M REVERSING IT!

for i, j in zip(idx2, idx1):
    MAC.at[i, 'Selection Method'] += ' / ' + lens.at[j, 'Selection Method']
    MAC.at[i, 'Paper(s)'] += ' ; ' + lens.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + lens.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + lens.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != lens.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + lens.at[j, 'System Type']
    if MAC.at[i, 'Confirmation Method'] != lens.at[j, 'Confirmation Method']:
        MAC.at[i, 'Confirmation Method'] += ' / ' + str(lens.at[j, 'Confirmation Method'])
    #MAC.at[i, 'Notes'] += '. ' + str(lens.at[j, 'Notes'])

for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Selection Method'] += ' / ' + lens.at[j, 'Selection Method']
    MAC.at[i, 'Paper(s)'] += ' ; ' + lens.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + lens.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + lens.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != lens.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + lens.at[j, 'System Type']
    if MAC.at[i, 'Confirmation Method'] != lens.at[j, 'Confirmation Method']:
        MAC.at[i, 'Confirmation Method'] += ' / ' + str(lens.at[j, 'Confirmation Method'])
    #MAC.at[i, 'Notes'] += '. ' + str(lens.at[j, 'Notes'])

# dropping the matching indices from the lens table
lens.drop(idx1, axis=0, inplace=True)
lens.drop(idx1_2, axis=0, inplace=True)
lens.reset_index(drop=True, inplace=True)
#

# now trying out a match tolerance of 1''
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(lens,MAC,1)
##tunique, tmatches, idx1, idx2 
#
print(len(idx1),len(idx1_2)) #

for i in idx1:
    if i in idx1_2:
        print('Stop, there is a duplicate match.')
        break

for i, j in zip(idx2, idx1):
    MAC.at[i, 'Selection Method'] += ' / ' + lens.at[j, 'Selection Method']
    MAC.at[i, 'Paper(s)'] += ' ; ' + lens.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + lens.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + lens.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != lens.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + lens.at[j, 'System Type']
    if MAC.at[i, 'Confirmation Method'] != lens.at[j, 'Confirmation Method']:
        MAC.at[i, 'Confirmation Method'] += ' / ' + str(lens.at[j, 'Confirmation Method'])
    #MAC.at[i, 'Notes'] += '. ' + str(lens.at[j, 'Notes'])

for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Selection Method'] += ' / ' + lens.at[j, 'Selection Method']
    MAC.at[i, 'Paper(s)'] += ' ; ' + lens.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + lens.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + lens.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != lens.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + lens.at[j, 'System Type']
    if MAC.at[i, 'Confirmation Method'] != lens.at[j, 'Confirmation Method']:
        MAC.at[i, 'Confirmation Method'] += ' / ' + str(lens.at[j, 'Confirmation Method'])
    #MAC.at[i, 'Notes'] += '. ' + str(lens.at[j, 'Notes'])

# dropping the matching indices from the lens table
lens.drop(idx1, axis=0, inplace=True)
lens.drop(idx1_2, axis=0, inplace=True)
lens.reset_index(drop=True, inplace=True)

# now trying out a match tolerance of 5''
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(lens,MAC,5)
##tunique, tmatches, idx1, idx2 
#
print(len(idx1),len(idx1_2)) #

for i in idx1:
    if i in idx1_2:
        print('Stop, there is a duplicate match.')
        break

## now trying out a match tolerance of 40''
#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(lens,MAC,50)
###tunique, tmatches, idx1, idx2 
#
#print(len(idx1),len(idx1_2)) # there is a match within 50'' between a target from SPiniello2018 and Hennawi
# it is unclear if Spiniello2018s target is the same as Hennawi's; there is a photoetric object near the position \
# of the Spiniello2018 target that is closer than the object from Hennawis table. If only Spiniello had more \
# precise coordinates...

# We're going to assume they're the distinct objects, but we may come back to this later.
     

# WE NEED TO COME BACK AND REPLACE PHOT Z'S FROM MYERS WITH TEH SPEC Z'S OF LATER PAPERS LIKE EFTEK2017        
        
        


17 17
9 6
0 0


In [13]:
#MAC

In [14]:
# now checking our RA1 only objects from MAC against the lens table (with RA2s and without RA2s)
lensnora2 = lens[lens['RA2']=='-99']
lenswithra2 = lens[lens['RA2']!='-99']

# first matching against the lens table with RA2s
# now trying out a match tolerance of 5''
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(MACnora2,lenswithra2,5)

print(len(idx1),len(idx1_2)) # out to 60'', there are only two matches. \
# You can find the matches using only a 5'' tolerance

# we will defer to the Inada entries since there are distinct values 
for i, j in zip(idx1_2, idx2_2):
    MACnora2.at[i, 'Paper(s)'] =lenswithra2.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = lenswithra2.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = lenswithra2.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    #MACnora2.at[i, 'Notes'] = lenswithra2.at[j, 'Notes'] + ' ' + MACnora2.at[i, 'Notes']
    MACnora2.at[i, 'Selection Method'] += ' / ' + lenswithra2.at[j, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] = lenswithra2.at[j, 'Confirmation Method'] #lenswithra2.at[j, 'Confirmation Method'] + 
    #MACnora2.at[i, 'Literature Name'] = lenswithra2.at[j, 'Literature Name']
    MACnora2.at[i, 'Name1'] = lenswithra2.at[j, 'Name1'] + ' / ' + MACnora2.at[i, 'Name2']
    MACnora2.at[i, 'Name2'] = lenswithra2.at[j, 'Name2'] + ' / ' + MACnora2.at[i, 'Name1']
    MACnora2.at[i, 'z1'] = lenswithra2.at[j, 'z1'] # fixing the NaN value in the double-peaked table
    MACnora2.at[i, 'z2'] = lenswithra2.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = lenswithra2.at[j, 'z1_type']
    MACnora2.at[i, 'z2_type'] = lenswithra2.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = lenswithra2.at[j, 'RA1']
    MACnora2.at[i, 'RA2'] = lenswithra2.at[j, 'RA2']
    MACnora2.at[i, 'Dec1'] = lenswithra2.at[j, 'Dec1']
    MACnora2.at[i, 'Dec2'] = lenswithra2.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = lenswithra2.at[j, 'Equinox1']
    MACnora2.at[i, 'Equinox2'] = lenswithra2.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = lenswithra2.at[j, 'Coordinate_waveband1']
    MACnora2.at[i, 'Coordinate_waveband2'] = lenswithra2.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = lenswithra2.at[j, 'Coordinate_Source1']
    MACnora2.at[i, 'Coordinate_Source2'] = lenswithra2.at[j, 'Coordinate_Source2']
    MACnora2.at[i, 'Sep'] = lenswithra2.at[j, 'Sep']


# dropping the matching indices from the lens table
lenswithra2.drop(idx2_2, axis=0, inplace=True)
lenswithra2.reset_index(drop=True, inplace=True)
#
#
## now matching against the lens table without RA2s
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(MACnora2,lensnora2,60)

#print(len(idx1),len(idx2))
# no matches out to 60''. We are good to clean up these tables and combine the two MAC tables and the lens tables back together

# verified
#tmatches_2


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['Table_flag'] = 'Table2'


0 2


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1782938724.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lenswithra2.drop(idx2_2, axis=0, inplace=True)


In [15]:
# Now we are concatenating the previous tables together to generate the main table again
MAC = pd.concat([MAC,MACnora2,lenswithra2,lensnora2])
MAC.reset_index(drop=True, inplace=True)

In [16]:
#MAC

In [17]:
# matching the binary candidates based on periodicity against the MAC

# first separating MAC into two tables, one with RA2s and one without
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']

# matching against the first table (with RA2s)

# FOR THIS CASE: IDX1 IS FOR periodicity AND IDX2 IS FOR MAC
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(periodicity,MAC,2)

#print(len(idx1), len(idx1_2)) # there is 1 match between the Graham+ table the MAC, where AGN 1 was matched \
# as having a periodic binary candidate

for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + periodicity.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + periodicity.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + periodicity.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != periodicity.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + periodicity.at[j, 'System Type']
    MAC.at[i, 'Notes'] += 'AGN 1 selected as a candidate binary based on periodicity.'
    MAC.at[i, 'Selection Method'] += ' / ' + periodicity.at[j, 'Selection Method']

# dropping the matching indices from the periodicity table
periodicity.drop(idx1, axis=0, inplace=True)
periodicity.reset_index(drop=True, inplace=True)

# and now matching against the second table (without RA2s)
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(periodicity,MACnora2,60)

#print(len(idx1), len(idx1_2)) # there are no additional matches

# this cell is verified now



In [18]:
# now concatenating all of the tables back together
MAC = pd.concat([MAC,MACnora2,periodicity])
MAC.reset_index(drop=True, inplace=True)


In [19]:
#MAC


In [20]:
# matching here the offset AGN tables to the MAC

# first separating MAC into two tables, one with RA2s and one without
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']

# matching against the first table (with RA2s)
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(offset,MAC,1)

print(len(idx1),len(idx1_2))

for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + offset.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + offset.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + offset.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != offset.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + offset.at[j, 'System Type']
    MAC.at[i, 'Notes'] += 'AGN 1 selected as a offset AGN candidate.'
    MAC.at[i, 'Selection Method'] += ' / ' + offset.at[j, 'Selection Method']
    
for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + offset.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + offset.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + offset.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != offset.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + offset.at[j, 'System Type']
    MAC.at[i, 'Notes'] += 'AGN 2 selected as a offset AGN candidate.'
    MAC.at[i, 'Selection Method'] += ' / ' + offset.at[j, 'Selection Method']

    
# dropping the matching indices from the offset table
offset.drop(idx1, axis=0, inplace=True)
offset.drop(idx1_2, axis=0, inplace=True)
offset.reset_index(drop=True, inplace=True)


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(offset,MAC,3)

#print(len(idx1),len(idx1_2)) # only one match out to 60'' for this second iteration. Same match found \
# in both tmatches tables. Just using idx1 in this case...

for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + offset.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + offset.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + offset.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != offset.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + offset.at[j, 'System Type']
    MAC.at[i, 'Notes'] += 'AGN 1 selected as a offset AGN candidate.'
    MAC.at[i, 'Selection Method'] += ' / ' + offset.at[j, 'Selection Method']

offset.drop(idx1, axis=0, inplace=True)
#offset.drop(idx1_2, axis=0, inplace=True) # don't need this, because idx1=idx1_2
offset.reset_index(drop=True, inplace=True)


# now matching against the second table (without RA2s)
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(offset,MACnora2,10)

print(len(idx1)) # for this matching setup out to 60''

# verified


7 5
1


In [21]:
# we can now join the tables back together...
MAC = pd.concat([MAC,MACnora2,offset])
MAC.reset_index(drop=True, inplace=True)


In [22]:
# this is a note to go and check what went wrong with Kim+2020 in the double=peaked notebook...





In [23]:
# now matching the offset broad line/unusual broad line binary candidates against the MAC

# first separating MAC into two tables, one with RA2s and one without
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']

# matching against the first table (with RA2s)
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(binary_BL,MAC,0.25)

print(len(idx1),len(idx1_2))

# we only need idx1_2 and idx2_2 here because there are no idx1 and idx2 values at this match tolerance
for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + binary_BL.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + binary_BL.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + binary_BL.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != binary_BL.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + binary_BL.at[j, 'System Type']
    MAC.at[i, 'Notes'] += 'AGN 2 selected as a binary candidate based on radial velocity shifts/extreme velocity binary_BLs.'
    MAC.at[i, 'Selection Method'] += ' / ' + binary_BL.at[j, 'Selection Method']

# dropping the matching indices from the binary_BL table
binary_BL.drop(idx1, axis=0, inplace=True)
binary_BL.drop(idx1_2, axis=0, inplace=True)
binary_BL.reset_index(drop=True, inplace=True)


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(binary_BL,MAC,5)

print(len(idx1),len(idx1_2))


# we only need idx1_2 and idx2_2 here because there are no idx1 and idx2 values at this match tolerance
for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + binary_BL.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + binary_BL.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + binary_BL.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != binary_BL.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + binary_BL.at[j, 'System Type']
    #MAC.at[i, 'Notes'] += 'AGN 2 selected as a binary candidate based on radial velocity shifts/extreme velocity binary_BLs.'
    # notes aren't needed; these objects were flagged as binaries
    MAC.at[i, 'Selection Method'] += ' / ' + binary_BL.at[j, 'Selection Method']
    #MAC.at[i, 'Name1'] += ' ; ' + binary_BL.at[j, 'Name1']
    # --> we need to come back and overwrite the name since Tsalamantza+ and Liu+ looked at these objects first
    
# dropping the matching indices from the binary_BL table
binary_BL.drop(idx1, axis=0, inplace=True)
#binary_BL.drop(idx1_2, axis=0, inplace=True) # not needed since idx1=idx1_2
binary_BL.reset_index(drop=True, inplace=True)


# and now the final table needed matching...
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(binary_BL,MACnora2,10)

print(len(idx1)) 

for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] += ' ; ' + binary_BL.at[j, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] += ' ; ' + binary_BL.at[j, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] += ' ; ' + binary_BL.at[j, 'DOI(s)']
    if MACnora2.at[i, 'System Type'] != binary_BL.at[j, 'System Type']:
        MACnora2.at[i, 'System Type'] += ' / ' + binary_BL.at[j, 'System Type']
    #MACnora2.at[i, 'Notes'] += 'AGN 2 selected as a binary candidate based on radial velocity shifts/extreme velocity binary_BLs.'
    # notes aren't needed; these objects were flagged as binaries
    MACnora2.at[i, 'Selection Method'] += ' / ' + binary_BL.at[j, 'Selection Method']
    #MACnora2.at[i, 'Notes'] += binaryBL.at[j, 'Notes']
    # come back and add notes!!!
    
# dropping the matching indices from the binary_BL table
binary_BL.drop(idx1, axis=0, inplace=True)
#binary_BL.drop(idx1_2, axis=0, inplace=True) # not needed since idx1=idx1_2
binary_BL.reset_index(drop=True, inplace=True)

# verified


0 1
2 2
7


In [24]:
# and now concatenating these tables back together
MAC = pd.concat([MAC,MACnora2,binary_BL])
MAC.reset_index(drop=True, inplace=True)


In [25]:
#MAC

In [26]:
# now matching the X-shaped radio sources against the MAC

# first separating MAC into two tables, one with RA2s and one without
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']


In [27]:

# matching against the first table (with RA2s)
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(xshape,MAC,0.05)

print(len(idx1),len(idx1_2))


# doesnt matter if we use idx1_2 and idx2_2 or idx1 and idx2 at this match tolerance
for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + xshape.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + xshape.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + xshape.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != xshape.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + xshape.at[j, 'System Type']
    if MAC.at[i, 'Notes'] == '-99':
        MAC.at[i, 'Notes'] = xshape.at[j, 'Notes']
    else:
        MAC.at[i, 'Notes'] += xshape.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] += ' / ' + xshape.at[j, 'Selection Method']

# dropping the matching indices from the xshape table
xshape.drop(idx1, axis=0, inplace=True)
#xshape.drop(idx1_2, axis=0, inplace=True) # idx1=dix1_2 here, so no need
xshape.reset_index(drop=True, inplace=True)





#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(MAC,xshape,10)



# do stuff here for this match....


#print(len(tmatches))
# there is one match, between Comerford+2014 and Yang+2019. This object, J1140+1743 is in SDSS

0 0


In [28]:
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(xshape,MAC,0.1)

print(len(idx1),len(idx1_2))

for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + xshape.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + xshape.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + xshape.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != xshape.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + xshape.at[j, 'System Type']
    #if MAC.at[i, 'Notes'] == '-99':
    #    MAC.at[i, 'Notes'] = xshape.at[j, 'Notes']
    #else:
    MAC.at[i, 'Notes'] += ' AGN 2 identified as a recoil candidate due to X-shaped radio source.'
    MAC.at[i, 'Selection Method'] += ' / ' + xshape.at[j, 'Selection Method']

# dropping the matching indices from the xshape table
xshape.drop(idx1_2, axis=0, inplace=True)
#xshape.drop(idx1_2, axis=0, inplace=True) # idx1=dix1_2 here, so no need
xshape.reset_index(drop=True, inplace=True)





0 1


In [29]:
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(xshape,MAC,60)

print(len(idx1),len(idx1_2))

# doesnt matter if we use idx1_2 and idx2_2 or idx1 and idx2 at this match tolerance
for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + xshape.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + xshape.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + xshape.at[j, 'DOI(s)']
    if MAC.at[i, 'System Type'] != xshape.at[j, 'System Type']:
        MAC.at[i, 'System Type'] += ' / ' + xshape.at[j, 'System Type']
    if MAC.at[i, 'Notes'] == '-99':
        MAC.at[i, 'Notes'] = xshape.at[j, 'Notes'] + '. Identified as a recoil candidate due to X-shaped radio source.'
    else:
        MAC.at[i, 'Notes'] += ' AGN 2 identified as a recoil candidate due to X-shaped radio source.'
    MAC.at[i, 'Selection Method'] += ' / ' + xshape.at[j, 'Selection Method']

# dropping the matching indices from the xshape table
xshape.drop(idx1_2, axis=0, inplace=True)
#xshape.drop(idx1_2, axis=0, inplace=True) # idx1=dix1_2 here, so no need
xshape.reset_index(drop=True, inplace=True)



1 1


In [30]:
# now finally matching just against the single RA2 column

tunique, tmatches, idx1, idx2 = match_tables_fib_singles(xshape,MACnora2,60)

print(len(idx1)) 

for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] += ' ; ' + xshape.at[j, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] += ' ; ' + xshape.at[j, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] += ' ; ' + xshape.at[j, 'DOI(s)']
    if MACnora2.at[i, 'System Type'] != xshape.at[j, 'System Type']:
        MACnora2.at[i, 'System Type'] += ' / ' + xshape.at[j, 'System Type']
    if MACnora2.at[i, 'Notes'] == '-99':
        MACnora2.at[i, 'Notes'] = xshape.at[j, 'Notes'] + '. Identified as a recoil candidate due to X-shaped radio source.'
    else:
        MACnora2.at[i, 'Notes'] += ' Identified as a recoil candidate due to X-shaped radio source.'
    MACnora2.at[i, 'Selection Method'] += ' / ' + xshape.at[j, 'Selection Method']


# dropping the matching indices from the xshape table
xshape.drop(idx1, axis=0, inplace=True)
#xshape.drop(idx1_2, axis=0, inplace=True) # not needed since idx1=idx1_2
xshape.reset_index(drop=True, inplace=True)

# verified


9


In [31]:
# and combining the tables together again...
# this includes the MAC and the X-shaped radio sources that didn't match before...
MAC = pd.concat([MAC,MACnora2,xshape])
MAC.reset_index(drop=True, inplace=True)


In [32]:
# now matching the varstrometry table against the MAC

# first separating MAC into two tables, one with RA2s and one without
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']


In [33]:
# matching against the MAC table with RA2 vals
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(varstr,MAC,60)

print(len(idx1),len(idx1_2)) # no matches here even out to 60''

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(varstr,MACnora2,60)

print(len(tmatches))

for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] += ' ; ' + varstr.at[j, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] += ' ; ' + varstr.at[j, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] += ' ; ' + varstr.at[j, 'DOI(s)']
    if MACnora2.at[i, 'System Type'] != varstr.at[j, 'System Type']:
        MACnora2.at[i, 'System Type'] += ' / ' + varstr.at[j, 'System Type']
    if MACnora2.at[i, 'Notes'] == '-99':
        MACnora2.at[i, 'Notes'] = 'Identified as dual or possible binary based on vastrometry.'
    else:
        MACnora2.at[i, 'Notes'] += ' Identified as dual or possible binary based on vastrometry.'
    # no notes needed here
    MACnora2.at[i, 'Selection Method'] += ' / ' + varstr.at[j, 'Selection Method']


# dropping the matching indices from the varstr table
varstr.drop(idx1, axis=0, inplace=True)
#varstr.drop(idx1_2, axis=0, inplace=True) # not needed since idx1=idx1_2
varstr.reset_index(drop=True, inplace=True)

# no matches between the MAC so far and the varstrometry tables!

#verified 

0 0
14


In [34]:
tmatches

,level_0,index,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes,RA,Dec,Table_flag,SDSS1,rmag1,SDSS2,rmag2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Sep(kpc),dV_rwp,z,Coordinate_Source,ra1_sexagesimal,dec1_sexagesimal,ra2_sexagesimal,dec2_sexagesimal,Name,Vmed,r,Coordinate_waveband,Equinox
0,28,28,085122.37+472249.0,-99,-99.0,-99,-99,-99,8:51:22.33,47:22:49.81,132.843037,47.380502,8:51:22.33,47:22:49.81,132.843037,47.380502,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.0,-99,Varstrometry / Optical Imaging,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,-99,-99,-99,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29,29,085122.37+472249.0,-99,-99.0,-99,-99,-99,8:51:22.40,47:22:48.64,132.843328,47.380178,8:51:22.40,47:22:48.64,132.843328,47.380178,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.0,-99,Varstrometry / Optical Imaging,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,-99,-99,-99,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34,34,133243.66+343300.6,-99,-99.0,-99,-99,-99,13:32:43.62,34:33:00.85,203.181733,34.550236,13:32:43.62,34:33:00.85,203.181733,34.550236,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.0,-99,Varstrometry / Optical Imaging,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,-99,-99,-99,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35,35,133243.66+343300.6,-99,-99.0,-99,-99,-99,13:32:43.68,34:33:00.26,203.182008,34.550073,13:32:43.68,34:33:00.26,203.182008,34.550073,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.0,-99,Varstrometry / Optical Imaging,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,-99,-99,-99,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,45,45,155259.18+230104.9,-99,-99.0,-99,-99,-99,15:52:59.22,23:01:04.93,238.246757,23.018037,15:52:59.22,23:01:04.93,238.246757,23.018037,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.0,-99,Varstrometry / Optical Imaging,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,-99,-99,-99,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,46,46,155259.18+230104.9,-99,-99.0,-99,-99,-99,15:52:59.17,23:01:04.98,238.246530,23.018050,15:52:59.17,23:01:04.98,238.24653,23.01805,J2000,Optical,GAIA DR2 Barycenter Coordinates,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99.0,-99,Varstrometry / Optical Imaging,-99,Hwang+2020,2020ApJ...888...73H,https://doi.org/10.3847/1538-4357/ab5c1a,-99,-99,-99,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,70,70,J1006+3454,-99,0.099,-99,-99,-99,10:06:01.7503,34:54:10.401,151.507293,34.902889,10:06:01.7503,34:54:10.401,151.507293,34.902889,J2000,Radio,ICRF,Dual AGN Candidate / Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.217,-99,Radio-Optical Positional Offsets / Radio Imagi...,-99,Orosz+2013,2013A&A...553A..13O,https://doi.org/10.1051/0004-6361/201321279,-99,10:06:01.7503,34:54:10.401,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2879,394,J100601.74+345410.5,J100601.74+345410.5,0.09938,-99.0,spec,-99,10:06:01.74,+34:54:10.5,151.507250,34.902917,-99,-99,151.50725,34.902917,J2000,Optical,-99,Dual AGN Candidate,15.35,SDSS r-band,mag,15.35,SDSS r-band,mag,3.0,-99.0,Double-Peaked Optical

In [35]:
# and now combing together the MAC, MACnora2, and the varstrometry table
MAC = pd.concat([MAC,MACnora2,varstr])
MAC.reset_index(drop=True, inplace=True)

In [36]:
MAC.to_csv('DR1_lessindivtargets.csv', sep=',', index=False)


In [37]:
# now finally adding in the individual targets table and working to match it against the full table
indiv = pd.read_csv('MAC_individualtargets_singleline.csv', sep=',')
# Convert NaNs and integers to empty strings
indiv['Notes'] = indiv['Notes'].apply(lambda x: '' if pd.isna(x) or isinstance(x, int) else x)
# Replace '-99' with empty strings (or another placeholder)
indiv['Notes'] = indiv['Notes'].replace('-99', '')

MAC = pd.read_csv('DR1_lessindivtargets.csv', sep=',')
MAC.reset_index(drop=True, inplace=True)

indivnora = indiv[indiv['RA1']=='-99']
#indivnora2 = indiv[indiv['RA2']]=='-99'
indiv = indiv[(indiv['RA1']!='-99')] #& (indiv['RA2']!='-99')

coordconvert = SkyCoord(ra = indiv['RA1'], dec = indiv['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
indiv['RA1_deg'] = coordconvert.ra.degree
indiv['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
#indiv['RA2'] = indiv['RA']
#indiv['Dec2'] = indiv['Dec']

#indiv['RA2_deg'] = indiv['RA1_deg']
#indiv['Dec2_deg'] = indiv['Dec1_deg']


# Convert NaNs and integers to empty strings
MAC['Notes'] = MAC['Notes'].apply(lambda x: '' if pd.isna(x) or isinstance(x, int) else x)

# Replace '-99' with empty strings (or another placeholder)
MAC['Notes'] = MAC['Notes'].replace('-99', '')


In [38]:
#MAC

In [39]:
# ra_decimal = 211.90487
# dec_decimal = 44.48227

# # Convert RA from degrees to hours
# ra_in_hours = ra_decimal / 15.0
# ra_angle = Angle(ra_in_hours, unit='hour')

# # Convert Dec
# dec_angle = Angle(dec_decimal, unit='deg')

# # Print out in sexagesimal format
# print(f"RA in Sexagesimal: {ra_angle.to_string(sep=':', pad=True)}")
# print(f"Dec in Sexagesimal: {dec_angle.to_string(sep=':', pad=True)}")

In [40]:

# Assuming indiv has been read already as:
# indiv = pd.read_csv('MAC_individualtargets_singleline.csv', sep=',')

def is_sexagesimal(s):
    # Check for the presence of ':' which indicates sexagesimal format
    return ':' in s

def check_coordinate_format(column, name, unit_type):
    for idx, entry in enumerate(column):
        str_entry = str(entry)
        if not is_sexagesimal(str_entry):
            # Convert the RA from degrees to hours if needed
            if unit_type == 'hourangle':
                entry = float(entry) / 15.0  # Convert degrees to hours for RA
                angle = Angle(entry, unit='hour')
            else:
                angle = Angle(entry, unit=unit_type)
                
            # If the entry is not in sexagesimal format, print the row number and the sexagesimal representation
            print(f"Decimal format found for {name} on line {idx + 2}: {str_entry}. Sexagesimal: {angle.to_string(sep=':', pad=True)}")

# Check the RA1 and Dec1 columns with their appropriate units
check_coordinate_format(indiv['RA1'], 'RA1', 'hourangle')
check_coordinate_format(indiv['Dec1'], 'Dec1', 'deg')


In [41]:
from astropy.coordinates import Angle

def check_invalid_angles(column, name):
    for idx, entry in enumerate(column):
        try:
            # Try to parse the entry as an angle
            Angle(entry, unit='hourangle') # adjust the unit as necessary
        except ValueError:
            # If there's a ValueError, print the problematic line number and entry
            print(f"Invalid {name} found on line {idx + 2} (considering header): {entry}")

# Check the RA1 and Dec1 columns for invalid entries
check_invalid_angles(indiv['RA1'], 'RA1')
check_invalid_angles(indiv['Dec1'], 'Dec1')



Invalid Dec1 found on line 3 (considering header): +26:36:09.893921938
Invalid Dec1 found on line 7 (considering header): +25:08:12.600227883
Invalid Dec1 found on line 9 (considering header): +48:58:49.426017
Invalid Dec1 found on line 10 (considering header): +32:22:34.372277976
Invalid Dec1 found on line 12 (considering header): +26:25:06.826057566
Invalid Dec1 found on line 13 (considering header): +48:16:41.113184700
Invalid Dec1 found on line 21 (considering header): +50:15:35.870275829
Invalid Dec1 found on line 24 (considering header): +79:46:17.129210139
Invalid Dec1 found on line 25 (considering header): +73:58:01.571908984
Invalid Dec1 found on line 26 (considering header): -34:34:19.121924786
Invalid Dec1 found on line 27 (considering header): -42:57:11.052847150
Invalid Dec1 found on line 28 (considering header): +35:51:25.658404975
Invalid Dec1 found on line 30 (considering header): +31:56:18.640
Invalid Dec1 found on line 31 (considering header): -32:49:08.6
Invalid Dec1

In [42]:
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']


In [43]:
# here's what we're going to do
# to make things a bit easier as far as matching goes and to give us some more control, we're going to parse the \
# individual target table by selection method, then match against the MAC, then modify the data, rinse, repeat

# this should make things a bit easier and it'll make it easier to catch oddities as well



In [44]:
# first, focusing on the easiest objects to match, the X-shaped radio sources...
# In the individual table, the selection method is flaged as: X-Shaped Radio Source
indiv_x = indiv[indiv['Selection Method']=='X-Shaped Radio Source']

# matching against the MAC table with RA2 vals
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_x,MAC,60)

print(len(idx1),len(idx1_2)) # 


# Here we're going to defer to the MAC table over the individual targets, but we'll add a couple extra notes, \
# like how Abell 1145 is also known as B1059+169 (used by Merritt+2002), or how B2 0828+32 (a recoil candidate) 
# and J083127.50+321926.9 (a dual AGN candidate based on double-peaks) are the same object and we need to include \
# both classes

# For now we're going to throw out the coordinates from the individual table, even though in some cases we know \
# the coordinate equinox and the waveband info. *We can comback and fix this later.*

for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_x.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + indiv_x.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + indiv_x.at[j, 'DOI(s)']
    #if MAC.at[i, 'System Type'] != indiv_x.at[j, 'System Type']:
    #    MAC.at[i, 'System Type'] += ' / ' + indiv_x.at[j, 'System Type']
    if MAC.at[i, 'Name1'] != indiv_x.at[j, 'Name1']:
        MAC.at[i, 'Name1'] += ' / ' + indiv_x.at[j, 'Name1']
    MAC.at[i, 'Notes'] += indiv_x.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] += ' / ' + indiv_x.at[j, 'Selection Method']
    if MAC.at[i, 'Name1'] == 'B1059+169 (Abell 1145)':
        MAC.at[i, 'Name1'] += 'Abell 1145 / B1059+169'
    
# dropping the matching indices from the indiv table
indiv_x.drop(idx1, axis=0, inplace=True)
#indiv_x.drop(idx1_2, axis=0, inplace=True)
indiv_x.reset_index(drop=True, inplace=True)


# matching again against the MAC table with RA2 vals to take care of the last remaining match in tmatches_2
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_x,MAC,60)

print(len(idx1),len(idx1_2)) # 


for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_x.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + indiv_x.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + indiv_x.at[j, 'DOI(s)']
    if MAC.at[i, 'Name2'] == 'J080535.00+240950.3':
        MAC.at[i, 'Name2'] += 'J080535.00+240950.3 / 3C 192'

    
# dropping the matching indices from the indiv table
indiv_x.drop(idx1_2, axis=0, inplace=True)
#indiv_x.drop(idx1_2, axis=0, inplace=True)
indiv_x.reset_index(drop=True, inplace=True)


# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_x,MACnora2,60)

print(len(idx1)) #,len(idx)) # 


for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] += ' ; ' + indiv_x.at[j, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] += ' ; ' + indiv_x.at[j, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] += ' ; ' + indiv_x.at[j, 'DOI(s)']
    if MACnora2.at[i, 'System Type'] != indiv_x.at[j, 'System Type']:
        MACnora2.at[i, 'System Type'] += ' / ' + indiv_x.at[j, 'System Type']
    if MACnora2.at[i, 'Name1'] != indiv_x.at[j, 'Name1']:
        MACnora2.at[i, 'Name1'] += ' / ' + indiv_x.at[j, 'Name1']
    MACnora2.at[i, 'Notes'] += indiv_x.at[j, 'Notes']
    MACnora2.at[i, 'Selection Method'] += ' / ' + indiv_x.at[j, 'Selection Method']
    if MACnora2.at[i, 'Name1'] == 'J094124+394441':
        MACnora2.at[i, 'RA1'] = indiv_x.at[j,'RA1']
        MACnora2.at[i, 'Dec1'] = indiv_x.at[j,'Dec1']
        MACnora2.at[i, 'Equinox1'] = indiv_x.at[j,'Equinox1']
# dropping the matching indices from the indiv table
indiv_x.drop(idx1, axis=0, inplace=True)
#indiv_x.drop(idx1_2, axis=0, inplace=True)
indiv_x.reset_index(drop=True, inplace=True)


# now just to check and make sure there are no remaining matches....
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_x,MAC,60)

print(len(idx1),len(idx1_2)) # 

tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_x,MACnora2,60)

print(len(idx1)) #,len(idx)) # 

# verified



/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


9 10


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1451110174.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_x.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


0 1


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1451110174.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_x.drop(idx1_2, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

2


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1451110174.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_x.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

0 0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


0


In [45]:
print(len(indiv_x))

1


In [46]:
## combining now and then breaking the MAC back apart...
#MAC = pd.concat([MAC,MACnora2,indiv_x])
#MAC = pd.concat([MAC,MACnora2,indiv_1])
#MACnora2 = MAC[MAC['RA2']=='-99']
#MAC = MAC[MAC['RA2']!='-99']


In [47]:
indiv = indiv[indiv['Selection Method']!='X-Shaped Radio Source']
print(len(indiv))

309


In [48]:
# Okay, now it gets tricky. There are a lot of objects from various sources

# We'll begin by carving this up by Paper name in the individual table
# Here we'll start with Farina+2011, Sandrinelli+2014, Lena+2018, 'Sanchez Alvaro+2007 ; Ellison+2007', \
# Pindor+2006
indiv_1 = indiv[(indiv['Paper(s)']=='Farina+2011') | (indiv['Paper(s)']=='Sandrinelli+2014') | (indiv['Paper(s)']=='Farina+2011 ; Sandrinelli+2014')] \
#              (indiv['Paper(s)']=='Lena+2018') | (indiv['Paper(s)']=='Sanchez Alvaro+2007 ; Ellison+2007') | \
#              (indiv['Paper(s)']=='Pindor+2006')]
print(len(indiv_1))

16


In [49]:
# matching against the MAC table with RA2 vals
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_1,MAC,60)

print(len(idx1),len(idx1_2)) # 

# first we'll take care of these duplicates in Farina+2011, Sandrinelli+2014, and Hennawi+2006

for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_1.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + indiv_1.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + indiv_1.at[j, 'DOI(s)']
    MAC.at[i, 'System Type'] += ' / ' + 'Dual AGN'
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MAC.at[i, 'Notes'] += indiv_1.at[j, 'Notes']
    if MAC.at[i, 'Selection Method'] == '-99':
        MAC.at[i, 'Selection Method'] = indiv_1.at[j, 'Selection Method']
    elif MAC.at[i, 'Selection Method'] == -99:
        MAC.at[i, 'Selection Method'] = indiv_1.at[j, 'Selection Method']
    else:
        MAC.at[i, 'Selection Method'] += ' / ' + indiv_1.at[j, 'Selection Method']
    if MAC.at[i, 'Confirmation Method'] == '-99':
        MAC.at[i, 'Confirmation Method'] = indiv_1.at[j, 'Confirmation Method']
    elif MAC.at[i, 'Confirmation Method'] == -99:
        MAC.at[i, 'Confirmation Method'] = indiv_1.at[j, 'Confirmation Method']
    else:
        MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_1.at[j, 'Confirmation Method']

# dropping the matching indices from the indiv table
indiv_1.drop(idx1, axis=0, inplace=True)
#indiv_1.drop(idx1_2, axis=0, inplace=True)
indiv_1.reset_index(drop=True, inplace=True)


# matching against the MAC table with RA2 vals
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_1,MAC,60)

print(len(idx1),len(idx1_2)) # 

for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_1.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + indiv_1.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + indiv_1.at[j, 'DOI(s)']
    MAC.at[i, 'System Type'] += ' / ' + 'Dual AGN'
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MAC.at[i, 'Notes'] += indiv_1.at[j, 'Notes']
    if MAC.at[i, 'Selection Method'] == '-99':
        MAC.at[i, 'Selection Method'] = indiv_1.at[j, 'Selection Method']
    elif MAC.at[i, 'Selection Method'] == -99:
        MAC.at[i, 'Selection Method'] = indiv_1.at[j, 'Selection Method']
    else:
        MAC.at[i, 'Selection Method'] += ' / ' + indiv_1.at[j, 'Selection Method']
    if MAC.at[i, 'Confirmation Method'] == '-99':
        MAC.at[i, 'Confirmation Method'] = indiv_1.at[j, 'Confirmation Method']
    elif MAC.at[i, 'Confirmation Method'] == -99:
        MAC.at[i, 'Confirmation Method'] = indiv_1.at[j, 'Confirmation Method']
    else:
        MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_1.at[j, 'Confirmation Method']

# dropping the matching indices from the indiv table
indiv_1.drop(idx1_2, axis=0, inplace=True)
#indiv_1.drop(idx1_2, axis=0, inplace=True)
indiv_1.reset_index(drop=True, inplace=True)


# matching against the MAC table with RA2 vals
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_1,MAC,60)

print(len(idx1),len(idx1_2)) # 


# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_1,MACnora2,60)

print(len(idx1)) #,len(idx)) # 
# no matches with the noRA2 table so well ignore that one for now


# I corrected these duplicates in the individual table rather than having to correct them in an unusual fashion here
#J074759.02+431805.3 --> Sandrinelli+2014
#SDSS J074759.03+431805.5 --> Farina+2011

#J085625.63+511137.0 --> Sandrinelli
#SDSS J085625.63+511137.0 --> Farina

#J082439.83+235720.3
#J084541.18+071050.3


# verified


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


5 6


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2061156572.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_1.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


0 1


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2061156572.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_1.drop(idx1_2, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

0 0
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


In [50]:
## concatenating the remaining objects now....
#MAC = pd.concat([MAC,MACnora2,indiv_1])
#
#MAC['RA2'] = MAC['RA2'].apply(lambda x: '-99' if pd.isna(x) or isinstance(x, int) else x)
#
#
#MACnora2 = MAC[MAC['RA2']=='-99']
#MAC = MAC[MAC['RA2']!='-99']

In [51]:
print(len(indiv_1))

10


In [52]:
indiv = indiv[(indiv['Paper(s)']!='Farina+2011') & (indiv['Paper(s)']!='Sandrinelli+2014') & \
             (indiv['Paper(s)']!='Farina+2011 ; Sandrinelli+2014')]
print(len(indiv))

#indiv_1 = indiv[(indiv['Paper(s)']=='Farina+2011') | (indiv['Paper(s)']=='Sandrinelli+2014') | (indiv['Paper(s)']=='Farina+2011 ; Sandrinelli+2014')] \
#              (indiv['Paper(s)']=='Lena+2018') | (indiv['Paper(s)']=='Sanchez Alvaro+2007 ; Ellison+2007') | \
#              (indiv['Paper(s)']=='Pindor+2006')]



293


In [53]:
indiv_2 = indiv[(indiv['Paper(s)']=='Lena+2018') | (indiv['Paper(s)']=='Sanchez Alvaro+2007') | \
              (indiv['Paper(s)']=='Pindor+2006')]

print(len(indiv_2))

3


In [54]:
# matching against the MAC table with RA2 vals
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_2,MAC,60)

print(len(idx1),len(idx1_2)) # 

# manually removed the overlapping Pindor+ object since I listed Pindor already with Hennawi+....

for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_2.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + indiv_2.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + indiv_2.at[j, 'DOI(s)']
    #MAC.at[i, 'System Type'] += 'Dual AGN'
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MAC.at[i, 'Notes'] += indiv_2.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] += ' / ' + indiv_2.at[j, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_2.at[j, 'Confirmation Method']

# dropping the matching indices from the indiv table
indiv_2.drop(idx1, axis=0, inplace=True)
#indiv_2.drop(idx1_2, axis=0, inplace=True)
indiv_2.reset_index(drop=True, inplace=True)


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_2,MAC,60)

print(len(idx1),len(idx1_2)) # 


# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_2,MACnora2,60)

print(len(idx1)) #,len(idx)) # 

for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] += ' ; ' + indiv_2.at[j, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] += ' ; ' + indiv_2.at[j, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] += ' ; ' + indiv_2.at[j, 'DOI(s)']
    MACnora2.at[i, 'System Type'] += ' / Dual AGN'
    MACnora2.at[i, 'Name1'] = indiv_2.at[j, 'Name1']
    MACnora2.at[i, 'Name2'] = indiv_2.at[j, 'Name2']
    MACnora2.at[i, 'z2'] = indiv_2.at[j, 'z2']
    MACnora2.at[i, 'z2_type'] = indiv_2.at[j, 'z2_type']
    MACnora2.at[i, 'Notes'] += indiv_2.at[j, 'Notes']
    MACnora2.at[i, 'Selection Method'] += ' / ' + indiv_2.at[j, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] += ' / ' + indiv_2.at[j, 'Confirmation Method']

indiv_2.drop(idx1, axis=0, inplace=True)
#indiv_2.drop(idx1_2, axis=0, inplace=True)
indiv_2.reset_index(drop=True, inplace=True)

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_2,MACnora2,60)

#print(len(idx1)) #,len(idx)) # 


# verified



/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


2 2


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/311387926.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_2.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


0 0
1


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.drop(labels=['level_0'], axis=1, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/311387926.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

In [55]:
# now remove these listings from the individual table:
indiv = indiv[(indiv['Paper(s)']!='Lena+2018') & (indiv['Paper(s)']!='Sanchez Alvaro+2007') & \
              (indiv['Paper(s)']!='Pindor+2006')]
print(len(indiv))

290


In [56]:
# COME BACK AND MAKE SURE THAT IF I DEFER FOR RA1 AND DEC 1 THAT I ALSO DO IT FOR THE DEGREE COORDINATES






In [57]:
# now we'll target some other authors:
# Mason+2000, green+2010, green+2011, Brotherton+1999 ; Kunert-Bajraszewska+2011 , Schneider+2000 ; Fukugita+2004
# Hewett ; Faure, Schechter (but only one match)
# 

# I need to do these all separately to handle them carefully...


indiv_3 = indiv[(indiv['Paper(s)']=='Mason+2000')]


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_3,MAC,60)

print(len(idx1),len(idx1_2)) # 


# we're deferring to Mason+2000 here in some cases instead of Inada...
# I'm deferring actually mostly to Inada. Previously I was using redshifts drawn from SImbad (which came from SDSS)
# however, at ~4.5'' separation, we are likely dealing with fiber spillover, which is why the redshifts measurede \
# by Inada are distinctly different than that determined from SDSS (Inada's redshifts yield a dV of ~3000 \
# km s^-1 and SDSS is only a few hundred km s^-1')

for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_3.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_3.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_3.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    #MAC.at[i, 'System Type'] = indiv_3.at[j,'Dual AGN']
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MAC.at[i, 'Notes'] = indiv_3.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_3.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    #MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_3.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_3.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_3.at[j, 'Name1']
    MAC.at[i, 'Name2'] = indiv_3.at[j, 'Name2']
    #MAC.at[i, 'z1'] = indiv_3.at[j, 'z1']
    #MAC.at[i, 'z2'] = indiv_3.at[j, 'z2']
    #MAC.at[i, 'RA1'] = indiv_3.at[j, 'RA1']
    #MAC.at[i, 'RA2'] = indiv_3.at[j, 'RA2']
    #MAC.at[i, 'Dec1'] = indiv_3.at[j, 'Dec1']
    #MAC.at[i, 'Dec2'] = indiv_3.at[j, 'Dec2']
    #MAC.at[i, 'Equinox1'] = indiv_3.at[j, 'Equinox1']
    #MAC.at[i, 'Equinox2'] = indiv_3.at[j, 'Equinox2']
    #MAC.at[i, 'Coordinate_waveband1'] = indiv_3.at[j, 'Coordinate_waveband1']
    #MAC.at[i, 'Coordinate_waveband2'] = indiv_3.at[j, 'Coordinate_waveband2']
    #MAC.at[i, 'Coordinate_Source1'] = indiv_3.at[j, 'Coordinate_Source1']
    #MAC.at[i, 'Coordinate_Source2'] = indiv_3.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_3.at[j, 'Sep']


# dropping the matching indices from the indiv table
indiv_3.drop(idx1, axis=0, inplace=True)
#indiv_3.drop(idx1_2, axis=0, inplace=True)
indiv_3.reset_index(drop=True, inplace=True)

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_3,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 


indiv_3 = indiv[(indiv['Paper(s)']=='Green+2010')]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_3,MAC,60)

print(len(idx1),len(idx1_2)) # 

# we're deferring to Green+2010...
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_3.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_3.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_3.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = indiv_3.at[j,'System Type']    
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MAC.at[i, 'Notes'] = indiv_3.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] += ' / ' + indiv_3.at[j, 'Selection Method'] 
    MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_3.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_3.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_3.at[j, 'Name1']
    MAC.at[i, 'Name2'] = indiv_3.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_3.at[j, 'z1']
    MAC.at[i, 'z2'] = indiv_3.at[j, 'z2']
    MAC.at[i, 'RA1'] = indiv_3.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_3.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_3.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_3.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_3.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_3.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_3.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_3.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_3.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_3.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_3.at[j, 'Sep']
    
# dropping the matching indices from the indiv table
indiv_3.drop(idx1, axis=0, inplace=True)
#indiv_3.drop(idx1_2, axis=0, inplace=True)
indiv_3.reset_index(drop=True, inplace=True)

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_3,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 


indiv_3 = indiv[(indiv['Paper(s)']=='Green+2011')]
# we'd actually already accounted for Green+2011's target being in the Eftek. table from 2017, so we only need to adjust a \
# few things (since we overwrote the Eftek. table previously)


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_3,MAC,60)

print(len(idx1),len(idx1_2)) # 

# we're deferring to Green+2010...
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_3.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_3.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_3.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = indiv_3.at[j,'System Type']    
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MAC.at[i, 'Notes'] = indiv_3.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_3.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = indiv_3.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_3.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_3.at[j, 'Name1']
    MAC.at[i, 'Name2'] = indiv_3.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_3.at[j, 'z1']
    MAC.at[i, 'z2'] = indiv_3.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_3.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_3.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_3.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_3.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_3.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_3.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_3.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_3.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_3.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_3.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_3.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_3.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_3.at[j, 'Sep']
    
# dropping the matching indices from the indiv table
indiv_3.drop(idx1, axis=0, inplace=True)
#indiv_3.drop(idx1_2, axis=0, inplace=True)
indiv_3.reset_index(drop=True, inplace=True)

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_3,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 


indiv_3 = indiv[(indiv['Paper(s)']=='Schneider+2000 ; Fukugita+2004')]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_3,MAC,60)

print(len(idx1),len(idx1_2)) # 

# here we're going to refer to Schneider+ and Fukugita+ over Hennawi+ since the other two came first
# double check how hennawi selects this object
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_3.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_3.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_3.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = 'Dual AGN'   
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MAC.at[i, 'Notes'] = indiv_3.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_3.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = indiv_3.at[j, 'Confirmation Method'] + ' / ' + MAC.at[i, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_3.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_3.at[j, 'Name1']
    MAC.at[i, 'Name2'] = indiv_3.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_3.at[j, 'z1']
    MAC.at[i, 'z2'] = indiv_3.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_3.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_3.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_3.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_3.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_3.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_3.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_3.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_3.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_3.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_3.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_3.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_3.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_3.at[j, 'Sep']
    
# dropping the matching indices from the indiv table
indiv_3.drop(idx1, axis=0, inplace=True)
#indiv_3.drop(idx1_2, axis=0, inplace=True)
indiv_3.reset_index(drop=True, inplace=True)


#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_3,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 


indiv_3 = indiv[(indiv['Paper(s)']=='Hewett+1989 ; Faure+2003')]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_3,MAC,60)

print(len(idx1),len(idx1_2)) # 

# here we're combine some of the information between Hewett, Faure, and Hennawi
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_3.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_3.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_3.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = 'Dual AGN'   
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MAC.at[i, 'Notes'] = indiv_3.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_3.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = indiv_3.at[j, 'Confirmation Method'] + ' / ' + MAC.at[i, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_3.at[j, 'Literature Name'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'Name1'] = indiv_3.at[j, 'Name1']
    MAC.at[i, 'Name2'] = indiv_3.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_3.at[j, 'z1']
    MAC.at[i, 'z2'] = indiv_3.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_3.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_3.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_3.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_3.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_3.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_3.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = 'J2000'
    MAC.at[i, 'Equinox2'] = 'J2000'
    MAC.at[i, 'Coordinate_waveband1'] = 'Optical'
    MAC.at[i, 'Coordinate_waveband2'] = 'Optical'
    MAC.at[i, 'Coordinate_Source1'] = 'SDSS'
    MAC.at[i, 'Coordinate_Source2'] = 'SDSS'
    MAC.at[i, 'Sep'] = indiv_3.at[j, 'Sep']
    
# dropping the matching indices from the indiv table
indiv_3.drop(idx1, axis=0, inplace=True)
#indiv_3.drop(idx1_2, axis=0, inplace=True)
indiv_3.reset_index(drop=True, inplace=True)


# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_3,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 



indiv_3 = indiv[(indiv['Paper(s)']=='Brotherton+1999 ; Kunert-Bajraszewska+2011')]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_3,MAC,60)

print(len(idx1),len(idx1_2)) # 

# here we're going to defer pretty much to Brotherton+ and Kunert-Bajraszewska+
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_3.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_3.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_3.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = indiv_3.at[j, 'System Type'] 
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MAC.at[i, 'Notes'] = indiv_3.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_3.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = indiv_3.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_3.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_3.at[j, 'Name1']
    MAC.at[i, 'Name2'] = indiv_3.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_3.at[j, 'z1']
    MAC.at[i, 'z2'] = indiv_3.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_3.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_3.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_3.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_3.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_3.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_3.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_3.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_3.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_3.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_3.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_3.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_3.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_3.at[j, 'Sep']
    
# dropping the matching indices from the indiv table
indiv_3.drop(idx1, axis=0, inplace=True)
#indiv_3.drop(idx1_2, axis=0, inplace=True)
indiv_3.reset_index(drop=True, inplace=True)


# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_3,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 


indiv_3 = indiv[(indiv['Paper(s)']=='Schecter+2017') & (indiv['Name1']=='WISE 0326-3122')]
# there is only one matching object from Schechter's work, so we're honing in on that one specifically and \
# leaving the others alone


#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_3,MAC,60)

#print(len(idx1),len(idx1_2)) # 


# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_3,MACnora2,60)
print(len(idx1)) #,len(idx)) # 


# here we're going to defer pretty much to Brotherton+ and Kunert-Bajraszewska+
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_3.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_3.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_3.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    MACnora2.at[i, 'System Type'] = indiv_3.at[j, 'System Type'] 
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MACnora2.at[i, 'Notes'] +=  ' ' + indiv_3.at[j, 'Notes']
    MACnora2.at[i, 'Selection Method'] = indiv_3.at[j, 'Selection Method'] + ' / ' + MACnora2.at[i, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] = indiv_3.at[j, 'Confirmation Method']
    MACnora2.at[i, 'Literature Name'] = indiv_3.at[j, 'Literature Name']
    MACnora2.at[i, 'Name1'] = indiv_3.at[j, 'Name1']
    MACnora2.at[i, 'Name2'] = indiv_3.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = indiv_3.at[j, 'z1']
    MACnora2.at[i, 'z2'] = indiv_3.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_3.at[j, 'z1_type']
    MACnora2.at[i, 'z2_type'] = indiv_3.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_3.at[j, 'RA1']
    MACnora2.at[i, 'RA2'] = indiv_3.at[j, 'RA2']
    MACnora2.at[i, 'Dec1'] = indiv_3.at[j, 'Dec1']
    MACnora2.at[i, 'Dec2'] = indiv_3.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_3.at[j, 'Equinox1']
    MACnora2.at[i, 'Equinox2'] = indiv_3.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_3.at[j, 'Coordinate_waveband1']
    MACnora2.at[i, 'Coordinate_waveband2'] = indiv_3.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_3.at[j, 'Coordinate_Source1']
    MACnora2.at[i, 'Coordinate_Source2'] = indiv_3.at[j, 'Coordinate_Source2']
    MACnora2.at[i, 'Sep'] = indiv_3.at[j, 'Sep']
    
# dropping the matching indices from the indiv table
indiv_3.drop(idx1, axis=0, inplace=True)
#indiv_3.drop(idx1_2, axis=0, inplace=True)
indiv_3.reset_index(drop=True, inplace=True)


print(len(indiv_3))



/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3350777378.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_3.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3350777378.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_3.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


...

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3350777378.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_3.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3350777378.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_3.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1
1
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3350777378.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_3.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3350777378.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

In [58]:
# now removing these individaul targets from the individual table...
indiv = indiv[~(indiv['Paper(s)']=='Mason+2000') & ~(indiv['Paper(s)']=='Green+2010') & \
              ~(indiv['Paper(s)']=='Green+2011') & ~(indiv['Name1']=='WISE 0326-3122') & \
              ~(indiv['Paper(s)']=='Brotherton+1999 ; Kunert-Bajraszewska+2011') & \
             ~(indiv['Paper(s)']=='Hewett+1989 ; Faure+2003') & \
              ~(indiv['Paper(s)']=='Schneider+2000 ; Fukugita+2004')]

print(len(indiv))



283


In [59]:
# now on to some other individual targets
# Teng+2012, Kharb+2020, Kharb+2019, Krause+2019, Koss+2012
# Barrows+2013, Wang+2010, Keel+2019, SUn+2016
# Benitez+2018, Spiniello+2018, Ciaramella+2004

indiv_4 = indiv[((indiv['Paper(s)']=='Teng+2012') & (indiv['Name1']=='J1352+1429E')) | \
                (indiv['Paper(s)']=='Kharb+2019') | (indiv['Paper(s)']=='Kharb+2020') | \
               ((indiv['Paper(s)']=='Krause+2019') & (((indiv['Name1']=='3C 207') | (indiv['Name1']=='3C 433')))) | \
                ((indiv['Paper(s)']=='Koss+2012') & (indiv['Name1']=='Mrk 248')) | \
               ((indiv['Paper(s)']=='Wang+2010') & (indiv['Name1']=='PKS B1345+125 NW')) | \
                ((indiv['Paper(s)']=='Keel+2019') & ((indiv['Name1']=='SDSS J141447.15-000013.1') | (indiv['Name1']=='SDSS J084810.10+351534.4'))) | \
                (indiv['Paper(s)']=='Sun+2016') | \
               (indiv['Paper(s)']=='Benitez+2018') | \
                ((indiv['Paper(s)']=='Ciaramella+2004') & (indiv['Name1']=='4C 40.24 (0945+408)')) | \
                ((indiv['Paper(s)']=='Barrows+2011') & (indiv['Name1']=='SDSSJ0932+0318'))]

#indiv_4

In [60]:
indiv_4 = indiv[((indiv['Paper(s)']=='Krause+2019') & (((indiv['Name1']=='3C 207') | (indiv['Name1']=='3C 433'))))]


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)

print(len(idx1),len(idx1_2)) # 

# here we're going to defer pretty much to the MAC except for the coordinates because those come from simbad/ICRF
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_4.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + indiv_4.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + indiv_4.at[j, 'DOI(s)']
    MAC.at[i, 'System Type'] += ' / ' + indiv_4.at[j, 'System Type'] 
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MAC.at[i, 'Notes'] +=  ' ' + indiv_4.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']

# dropping the matching indices from the indiv table
indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 

# here we're going to defer pretty much to Liu+2014 except for the coordinates because those come from simbad/ICRF
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] += ' ; ' + indiv_4.at[j, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] += ' ; ' + indiv_4.at[j, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] += ' ; ' + indiv_4.at[j, 'DOI(s)']
    MACnora2.at[i, 'Notes'] +=  ' ' + indiv_4.at[j, 'Notes']
    MACnora2.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method']
    MACnora2.at[i, 'Name1'] += ' / ' + indiv_4.at[j, 'Name1']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']

# dropping the matching indices from the indiv table
indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Paper(s)']=='Keel+2019') & ((indiv['Name1']=='SDSS J141447.15-000013.1') | (indiv['Name1']=='SDSS J084810.10+351534.4')))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)

print(len(idx1),len(idx1_2)) # 


# here we're going to defer mostly to Liu2011b
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = MAC.at[i, 'Paper(s)'] + ' ; ' + indiv_4.at[j, 'Paper(s)'] 
    MAC.at[i, 'BibCode(s)'] = MAC.at[i, 'BibCode(s)'] + ' ; ' + indiv_4.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = MAC.at[i, 'DOI(s)'] + ' ; ' + indiv_4.at[j, 'DOI(s)']
    MAC.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method'] 
    MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_4.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']

# dropping the matching indices from the indiv table
indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 


# here we're going to defer pretty much to Keel because they have long-slit spectra and examine two objects
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = MACnora2.at[i, 'Paper(s)'] + ' ; ' + indiv_4.at[j, 'Paper(s)'] 
    MACnora2.at[i, 'BibCode(s)'] = MACnora2.at[i, 'BibCode(s)'] + ' ; ' + indiv_4.at[j, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = MACnora2.at[i, 'DOI(s)'] + ' ; ' + indiv_4.at[j, 'DOI(s)']
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MACnora2.at[i, 'Notes'] +=  ' ' + indiv_4.at[j, 'Notes'] + ' . AGN 1 is double-peaked based on Ge+2012.'
    MACnora2.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method'] 
    #MAC.at[i, 'Confirmation Method'] = indiv_4.at[j, 'Confirmation Method']
    MACnora2.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    MACnora2.at[i, 'Name2'] = 'J084809.69+351532.1' #indiv_4.at[j, 'Name2'] # name comes from Keel+2019 but witout 'SDSS'
    MACnora2.at[i, 'z1'] = indiv_4.at[j, 'z1']
    MACnora2.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MACnora2.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MACnora2.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MACnora2.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MACnora2.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MACnora2.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MACnora2.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MACnora2.at[i, 'Sep'] = indiv_4.at[j, 'Sep']

# Added this note: AGN 1 is double-peaked
# dropping the matching indices from the indiv table
indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[(indiv['Paper(s)']=='Kharb+2019') | (indiv['Paper(s)']=='Kharb+2020')]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)

print(len(idx1),len(idx1_2)) # 

# here we'll defer back and forth between the different papers (Kharb+19,20, Comerford+2014, Liu+2011b)
# here we're using the idx values from the second matches table and being careful not to mix and match names
for i, j in zip(idx2_2, idx1_2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_4.at[j, 'Paper(s)'] 
    MAC.at[i, 'BibCode(s)'] += ' ; ' + indiv_4.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + indiv_4.at[j, 'DOI(s)']
    MAC.at[i, 'Notes'] +=  ' ' + indiv_4.at[j, 'Notes']
    MAC.at[i, 'System Type'] += ' / ' + indiv_4.at[j, 'System Type'] 
    MAC.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method'] 
    MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_4.at[j, 'Confirmation Method']
    MAC.at[i, 'Name1'] += ' / ' + indiv_4.at[j, 'Name1']
    MAC.at[i, 'Name2'] += ' / ' + indiv_4.at[j, 'Name2']

MAC.loc[(MAC['Name1'] == 'J140343.63+292045.3'), 'Notes'] += '. AGN 2 is double-peaked based on Kharb+2019.'
MAC.loc[(MAC['Name1'] == 'J140343.63+292045.3'), 'Name1'] = 'J140343.63+292045.3'
MAC.loc[(MAC['Name1'] == 'J140343.63+292045.3'), 'Name2'] = 'J140345.02+292144.0 / KISSR 434 A+B'
MAC.loc[(MAC['Name1'] == 'J124135.09+285036.5'), 'Notes'] += 'Upper limit on binary separation from Kharb+2020 is 4.8e-3.'
# Added this note: AGN 1 is double-peaked
# dropping the matching indices from the indiv table
#indiv_4.drop(idx1, axis=0, inplace=True)
indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))
# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 



indiv_4 = indiv[(indiv['Paper(s)']=='Sun+2016')]

#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
#print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 

# here we're use a mix of Sun+ and Liu+
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] += ' ; ' + indiv_4.at[j, 'Paper(s)'] 
    MACnora2.at[i, 'BibCode(s)'] += ' ; ' + indiv_4.at[j, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] += ' ; ' + indiv_4.at[j, 'DOI(s)']
    MACnora2.at[i, 'Notes'] +=  ' ' + indiv_4.at[j, 'Notes'] + '. Liu+ flagged AGN 1 as a binary candidate while Sun+ flagged the whole system as a dual AGN candidate/recoil candidate.'
    MACnora2.at[i, 'System Type'] += ' / ' + indiv_4.at[j, 'System Type'] 
    MACnora2.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method'] 
    MACnora2.at[i, 'Confirmation Method'] += ' / ' + indiv_4.at[j, 'Confirmation Method']
    MACnora2.at[i, 'Name1'] += ' / ' + indiv_4.at[j, 'Name1']
    MACnora2.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = indiv_4.at[j, 'z1']
    MACnora2.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MACnora2.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Paper(s)']=='Ciaramella+2004') & (indiv['Name1']=='4C 40.24 (0945+408)'))]
print(len(indiv_4))
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 

# here we're use a mix of Ciamarella and Barrows
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    MACnora2.at[i, 'Notes'] +=  ' ' + indiv_4.at[j, 'Notes'] + ' . Ciaramella+ argue for a periodic binary candidate in AGN 1.'
    MACnora2.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] + ' / ' + MACnora2.at[i, 'System Type']
    MACnora2.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MACnora2.at[i, 'Selection Method']
    #MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_4.at[j, 'Confirmation Method']
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MACnora2.at[i, 'Name1']
    MACnora2.at[i, 'Literature Name'] = '4C 40.24'
    #MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = 1.2486 # this is what Barrows+2013 had, but for some reason z1 is listed as a NaN value. I need to check into this
    #MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    #MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Paper(s)']=='Barrows+2011') & (indiv['Name1']=='SDSSJ0932+0318'))]

print(len(indiv_4))
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 

# here we're use a mix of Barrows and Tsalmantza
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' . ' + MAC.at[i, 'Notes']
    #MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] + ' / ' + MAC.at[i, 'System Type']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_4.at[j, 'Confirmation Method']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    #MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']
    #MAC.at[i, 'z2'] = indiv_4.at[j, 'z1'] # note we're just overwriting z2 with z1, but we'll end up getting rid of it anyway
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    #MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[(indiv['Paper(s)']=='Benitez+2018')]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 

# here we're use a mix of Benitez and Wang, McGurk, etc.
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] += ' ; ' + indiv_4.at[j, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] += ' ; ' + indiv_4.at[j, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] += ' ; ' + indiv_4.at[j, 'DOI(s)']
    MACnora2.at[i, 'Notes'] += ' ' + indiv_4.at[j, 'Notes']
    MACnora2.at[i, 'System Type'] += ' / ' + indiv_4.at[j, 'System Type']
    MACnora2.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] += ' / ' + indiv_4.at[j, 'Confirmation Method']
    MACnora2.at[i, 'Name1'] += ' / ' + indiv_4.at[j, 'Name1'] 
    #MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    #MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = 0.02346 # marking the redshift from Wang since it is a NaN value
    #MACnora2.at[i, 'z2'] = -99 #indiv_4.at[j, 'z1'] # note we're just overwriting z2 with z1, but we'll end up getting rid of it anyway
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    #MACnora2.at[i, 'z2_type'] = '-99' #indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Paper(s)']=='Koss+2012') & (indiv['Name1']=='Mrk 248'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 

# here we're use a mixing and matching stuff from koss and Liu
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_4.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + indiv_4.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + indiv_4.at[j, 'DOI(s)']
    MAC.at[i, 'Notes'] += ' ' + indiv_4.at[j, 'Notes']
    MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type']
    MAC.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = indiv_4.at[j, 'Confirmation Method']
    # here, we need to account for the fact that AGN1 and 2 are switched between Koss and Liu
    #MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] += ' / ' + indiv_4.at[j, 'Name2'] 
    MAC.at[i, 'Name2'] += ' / ' + indiv_4.at[j, 'Name1']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z2'] #
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z1'] #
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source2']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source1']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Paper(s)']=='Teng+2012') & (indiv['Name1']=='J1352+1429E'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 

# here we're use a mixing and matching stuff from Teng and Liu
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_4.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + indiv_4.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + indiv_4.at[j, 'DOI(s)']
    MAC.at[i, 'Notes'] += ' ' + indiv_4.at[j, 'Notes']
    #MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type']
    MAC.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_4.at[j, 'Confirmation Method']
    # here, we need to account for the fact that AGN1 and 2 are switched between Teng and Liu
    #MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] += ' / ' + indiv_4.at[j, 'Name2'] 
    MAC.at[i, 'Name2'] += ' / ' + indiv_4.at[j, 'Name1']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z2'] #
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z1'] #
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z1_type']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Paper(s)']=='Wang+2010') & (indiv['Name1']=='PKS B1345+125 NW'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 

# here we're use a mixing and matching stuff from Yuan and Wang+2010
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' . Yuan+ argues this is a double-peaked AGN and therefore a dual AGN candidate. ' + MAC.at[i, 'Notes']
    #MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    #MAC.at[i, 'Confirmation Method'] += ' / ' + indiv_4.at[j, 'Confirmation Method']
    # here, we need to account for the fact that AGN1 and 2 are switched between Teng and Liu
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'z1'] = 0.1204 #taking this from Yuan
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1
1


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2225293080.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2225293080.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


...

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2225293080.py:355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2225293080.py:390: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


0 0
1


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2225293080.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)


In [61]:
indiv = indiv[~((indiv['Paper(s)']=='Teng+2012') & (indiv['Name1']=='J1352+1429E')) & \
                ~(indiv['Paper(s)']=='Kharb+2019') & ~(indiv['Paper(s)']=='Kharb+2020') & \
               ~((indiv['Paper(s)']=='Krause+2019') & (((indiv['Name1']=='3C 207') | (indiv['Name1']=='3C 433')))) & \
                ~((indiv['Paper(s)']=='Koss+2012') & (indiv['Name1']=='Mrk 248')) & \
               ~((indiv['Paper(s)']=='Wang+2010') & (indiv['Name1']=='PKS B1345+125 NW')) & \
                ~((indiv['Paper(s)']=='Keel+2019') & ((indiv['Name1']=='SDSS J141447.15-000013.1') | (indiv['Name1']=='SDSS J084810.10+351534.4'))) & \
                ~(indiv['Paper(s)']=='Sun+2016') & \
               ~(indiv['Paper(s)']=='Benitez+2018') & \
                ~((indiv['Paper(s)']=='Ciaramella+2004') & (indiv['Name1']=='4C 40.24 (0945+408)')) & \
                ~((indiv['Paper(s)']=='Barrows+2011') & (indiv['Name1']=='SDSSJ0932+0318'))]


In [62]:
print(len(indiv))

270


In [63]:
# now moving on to the next chunk of individual papers
#'Literature Name'=='J084135.08+010156.2' 'Satyapal+2017 ; Pfeifle+2019a'
#'Literature Name'=='J09527.62+255257.2' 'McGurk+2011'
#'Literature Name'=='SDSS J1048+0055' 'Zhou+2004 ; Jaiswal+2019'
#'Literature Name'=='SDSS J1316+1753' 'Xu+2009'
#'Literature Name'=='SDSS J150243.1+111557' 'Fu+2011'
#'Literature Name'=='SDSS J171544.05+600835.7' 'Comerford+2011'

#indiv_4 = indiv[((indiv['Paper(s)']=='Satyapal+2017 ; Pfeifle+2019a') & (indiv['Literature Name']=='J084135.08+010156.2')) | \
#                ((indiv['Paper(s)']=='McGurk+2011') & (indiv['Literature Name']=='J09527.62+255257.2')) | \
#               ((indiv['Paper(s)']=='Zhou+2004 ; Jaiswal+2019') & (indiv['Literature Name']=='SDSS J1048+0055')) | \
#                ((indiv['Paper(s)']=='Xu+2009') & (indiv['Literature Name']=='SDSS J1316+1753')) | \
#               ((indiv['Paper(s)']=='Fu+2011') & (indiv['Literature Name']=='SDSS J150243.1+111557')) | \
#                ((indiv['Paper(s)']=='Comerford+2011') & (indiv['Literature Name']=='SDSS J171544.05+600835.7'))]



indiv_4 = indiv[((indiv['Paper(s)']=='Comerford+2015 ; Barrows+2016 ; Satyapal+2017 ; Pfeifle+2019a') & (indiv['Literature Name']=='J084135.08+010156.2'))]

#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
#print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 

# here we're going to defer pretty much to Pfeifle+ and Comerford+
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    #MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MACnora2.at[i, 'Notes'] +=  ' ' + indiv_4.at[j, 'Notes']
    MACnora2.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MACnora2.at[i, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] = indiv_4.at[j, 'Confirmation Method']
    MACnora2.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    MACnora2.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = indiv_4.at[j, 'z1']
    MACnora2.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MACnora2.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MACnora2.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MACnora2.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MACnora2.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MACnora2.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MACnora2.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MACnora2.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
# we also need to come back and add in the details from Comerford+2015

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Paper(s)']=='McGurk+2011') & (indiv['Literature Name']=='J09527.62+255257.2'))]

#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
#print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 

# here we're going mix and match as needed
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    #MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MACnora2.at[i, 'Notes'] +=  ' ' + indiv_4.at[j, 'Notes']
    MACnora2.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MACnora2.at[i, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] = indiv_4.at[j, 'Confirmation Method']
    MACnora2.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    MACnora2.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = indiv_4.at[j, 'z1']
    MACnora2.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MACnora2.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    #MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    #MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    #MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    #MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MACnora2.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    #MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    #MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Paper(s)']=='Zhou+2004 ; Jaiswal+2019') & (indiv['Literature Name']=='SDSS J1048+0055'))]
                
#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
#print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 


# here we're going mix and match as needed
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    MACnora2.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MACnora2.at[i, 'Notes'] +=  ' ' + indiv_4.at[j, 'Notes']
    MACnora2.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MACnora2.at[i, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] = indiv_4.at[j, 'Confirmation Method']
    MACnora2.at[i, 'Literature Name'] = MACnora2.at[i, 'Name1'] # fixed this issue on 24 jan 20. j previously used
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    #MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = 0.6427 #indiv_4.at[j, 'z1'] # fixing the NaN value in the double-peaked table
    #MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    #MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    #MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    #MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    #MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    #MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    #MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MACnora2.at[i, 'Sep'] = indiv_4.at[j, 'Sep'] #+ ' / ' + MACnora2.at[i, 'Sep']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Paper(s)']=='Xu+2009') & (indiv['Literature Name']=='SDSS J1316+1753'))]

#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
#print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 


# here we're going mix and match as needed
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    #MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    #if MAC.at[i, 'Name1'] != indiv.at[j, 'Name1']:
    #    MAC.at[i, 'Name1'] += ' / ' + indiv.at[j, 'Name1']
    MACnora2.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MACnora2.at[i, 'Notes']
    MACnora2.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MACnora2.at[i, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] = indiv_4.at[j, 'Confirmation Method'] + ' / ' + MACnora2.at[i, 'Confirmation Method']
    MACnora2.at[i, 'Literature Name'] = MACnora2.at[j, 'Name1']
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    #MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = indiv_4.at[j, 'z1'] # fixing the NaN value in the double-peaked table
    #MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    #MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    #MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    #MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    #MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    #MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    #MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MACnora2.at[i, 'Sep'] = indiv_4.at[j, 'Sep']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Paper(s)']=='Fu+2011') & (indiv['Literature Name']=='SDSS J150243.1+111557'))]

#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
#print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 

# here we're going to pretty much defer to Fu+2011
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    MACnora2.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    MACnora2.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MACnora2.at[i, 'Notes']
    MACnora2.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MACnora2.at[i, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] = indiv_4.at[j, 'Confirmation Method'] + ' / ' + MACnora2.at[i, 'Confirmation Method']
    MACnora2.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    MACnora2.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = indiv_4.at[j, 'z1'] # fixing the NaN value in the double-peaked table
    MACnora2.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MACnora2.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MACnora2.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MACnora2.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MACnora2.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MACnora2.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MACnora2.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MACnora2.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    MACnora2.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    MACnora2.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    MACnora2.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    MACnora2.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    MACnora2.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    MACnora2.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Paper(s)']=='Comerford+2011') & (indiv['Literature Name']=='SDSS J171544.05+600835.7'))]


#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
#print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 

# here we're going to mix and match between Comerford+ and the others
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    MACnora2.at[i, 'System Type'] += ' / ' + indiv_4.at[j, 'System Type'] 
    MACnora2.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MACnora2.at[i, 'Notes']
    MACnora2.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MACnora2.at[i, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] = MACnora2.at[i, 'Confirmation Method'] #indiv_4.at[j, 'Confirmation Method'] + 
    MACnora2.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    #MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = indiv_4.at[j, 'z1'] # fixing the NaN value in the double-peaked table
    #MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    #MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    #MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    #MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    #MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    #MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    #MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    #MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/460821458.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

1
0
1
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/460821458.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/460821458.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

1
0
1
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/460821458.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1
0
1
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/460821458.py:250: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/460821458.py:302: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

In [64]:
indiv = indiv[~((indiv['Paper(s)']=='Comerford+2015 ; Barrows+2016 ; Satyapal+2017 ; Pfeifle+2019a') & (indiv['Literature Name']=='J084135.08+010156.2')) & \
                ~((indiv['Paper(s)']=='McGurk+2011') & (indiv['Literature Name']=='J09527.62+255257.2')) & \
               ~((indiv['Paper(s)']=='Zhou+2004 ; Jaiswal+2019') & (indiv['Literature Name']=='SDSS J1048+0055')) & \
                ~((indiv['Paper(s)']=='Xu+2009') & (indiv['Literature Name']=='SDSS J1316+1753')) & \
               ~((indiv['Paper(s)']=='Fu+2011') & (indiv['Literature Name']=='SDSS J150243.1+111557')) & \
                ~((indiv['Paper(s)']=='Comerford+2011') & (indiv['Literature Name']=='SDSS J171544.05+600835.7'))]

print(len(indiv))

264


In [65]:
# Now on the next set of individual objects..

#'Literature Name'=='J114753.62+094552.0' 'Satyapal+2017 ; Pfeifle+2019a'
#'Literature Name'=='SDSS J1050+3456' 'Shields+2009'
#'Name1'=='NGC 326 Core 1 (N)' 'Xu+2009'
#'Name1'=='PKS 0235+023' 
#'Name1'=='IRAS 12112+0305 SW' 'Imanishi+2020'


indiv_4 = indiv[((indiv['Paper(s)']=='Satyapal+2017 ; Pfeifle+2019a') & (indiv['Literature Name']=='J114753.62+094552.0'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 

# here we're going to mix and match between 
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_4.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + indiv_4.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + indiv_4.at[j, 'DOI(s)']
    #MAC.at[i, 'System Type'] += indiv_4.at[j, 'System Type'] 
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method']
    #MAC.at[i, 'Confirmation Method'] = ' / ' + MAC.at[i, 'Confirmation Method'] #indiv_4.at[j, 'Confirmation Method'] + 
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] += ' / ' + indiv_4.at[j, 'Name1']
    #MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    # I'm deferring to liu for Name 2 here!
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1'] # fixing the NaN value in the double-peaked table
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    # Here I'm adopting the X-ray position for AGN 1
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    #MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    #MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    #MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    #MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    #MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    #MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

# FIX THE REDSHIFT/NAME2 POSITION IN THE INDIVIDUAL TABLE!!!!!

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Paper(s)']=='Shields+2009') & (indiv['Literature Name']=='SDSS J1050+3456'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 


# here we're going to mix and match
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    MACnora2.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] + ' / ' + MACnora2.at[i, 'System Type']
    MACnora2.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MACnora2.at[i, 'Notes']
    MACnora2.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MACnora2.at[i, 'Selection Method']
    MACnora2.at[i, 'Confirmation Method'] = MACnora2.at[i, 'Confirmation Method'] #indiv_4.at[j, 'Confirmation Method'] + 
    MACnora2.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    #MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = indiv_4.at[j, 'z1'] # fixing the NaN value in the double-peaked table
    #MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    #MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    #MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    #MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    #MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    #MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    #MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    #MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Name1']=='PKS 0235+023'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 

# here we're going to mix and match
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    MACnora2.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] + ' / ' + MACnora2.at[i, 'System Type']
    MACnora2.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MACnora2.at[i, 'Notes']
    MACnora2.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method']
    #MAC.at[i, 'Confirmation Method'] = ' / ' + MAC.at[i, 'Confirmation Method'] #indiv_4.at[j, 'Confirmation Method'] + 
    #MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MACnora2.at[i, 'Name1']
    #MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    #MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    #MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    #MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    #MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    #MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    #MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    #MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    #MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Name1']=='NGC 326 Core 1 (N)'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
print(len(idx1)) #,len(idx)) # 


# here we're going to largely defer to the individual listing sine there are individual coordinates to each galaxy
for i, j in zip(idx2, idx1):
    MACnora2.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MACnora2.at[i, 'Paper(s)']
    MACnora2.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MACnora2.at[i, 'BibCode(s)']
    MACnora2.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MACnora2.at[i, 'DOI(s)']
    MACnora2.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    MACnora2.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MACnora2.at[i, 'Notes'] + '. AGN 1 is an X-shaped radio source.'
    MACnora2.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method']
    #MAC.at[i, 'Confirmation Method'] = ' / ' + MAC.at[i, 'Confirmation Method'] #indiv_4.at[j, 'Confirmation Method'] + 
    #MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MACnora2.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    MACnora2.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MACnora2.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MACnora2.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MACnora2.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MACnora2.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MACnora2.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MACnora2.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MACnora2.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MACnora2.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MACnora2.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MACnora2.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MACnora2.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MACnora2.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MACnora2.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MACnora2.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MACnora2.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    MACnora2.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    MACnora2.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    MACnora2.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    MACnora2.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    MACnora2.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    MACnora2.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Paper(s)']=='Imanishi+2020') & (indiv['Name1']=='IRAS 12112+0305 SW'))]


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 


# here we're going to largely defer to the individual listing sine there are individual coordinates to each galaxy
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += indiv_4.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += indiv_4.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += indiv_4.at[j, 'DOI(s)']
    #MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    MAC.at[i, 'Notes'] += indiv_4.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] += ' / ' + indiv_4.at[j, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] += ' / ' + MAC.at[i, 'Confirmation Method'] #indiv_4.at[j, 'Confirmation Method'] + 
    MAC.at[i, 'Literature Name'] = MAC.at[i, 'Name1'] + ' / ' + indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] += ' / ' + indiv_4.at[j, 'Name1']
    MAC.at[i, 'Name2'] += ' / ' + indiv_4.at[j, 'Name2']
    #MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    #MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    #MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    #MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    #MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    #MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    #MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    #MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    #MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    #MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    #MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    #MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    #MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    #MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    #MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1806617685.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1
0
1


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1806617685.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

...

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1806617685.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

1 1
0
1


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/3289966650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1806617685.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

1 1
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1806617685.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)


In [66]:
#'Literature Name'=='J114753.62+094552.0' 'Satyapal+2017 ; Pfeifle+2019a'
#'Name1'=='J140700.40+282714.6' 
#'Literature Name'=='SDSS J1050+3456' 'Shields+2009'
#'Name1'=='NGC 326 Core 1 (N)' 'Xu+2009'
#'Name1'=='PKS 0235+023' 
#'Name1'=='IRAS 12112+0305 SW' 'Imanishi+2020'


indiv = indiv[~((indiv['Paper(s)']=='Satyapal+2017 ; Pfeifle+2019a') & (indiv['Literature Name']=='J114753.62+094552.0')) & \
               ~((indiv['Paper(s)']=='Shields+2009') & (indiv['Literature Name']=='SDSS J1050+3456')) & \
                ~((indiv['Name1']=='NGC 326 Core 1 (N)')) & \
               ~((indiv['Name1']=='PKS 0235+023')) & \
                ~((indiv['Paper(s)']=='Imanishi+2020') & (indiv['Name1']=='IRAS 12112+0305 SW'))]

print(len(indiv))

259


In [67]:
# "One final effort is all that remains"

#'Literature Name'=='4C+22.25'
#'Literature Name'=='HS 1216+5032A' 
#'Literature Name'=='J2142-4420'
#'Literature Name'=='PG 1302-2102'
#'Literature Name'=='OQ 208 / Mrk 668'
#'Literature Name'=='1635+267'
#'Literature Name'=='2345+007'
#'Literature Name'=='3C 227'
#'Literature Name'=='J1536+0441'
#'Literature Name'=='Mrk 266'
#'Literature Name'=='SDSS J0927+2943'


indiv_4 = indiv[((indiv['Literature Name']=='4C+22.25'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv_4,MACnora2,60)
#print(len(idx1)) #,len(idx)) # 

print(tmatches_1)
# here we're mixing and matching between Decarli+2010 and the work by Tsalmantz, Decarli, and others
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    #MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = str(indiv_4.at[j, 'Confirmation Method']) + ' / ' + str(MAC.at[i, 'Confirmation Method']) #indiv_4.at[j, 'Confirmation Method'] + 
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name1']
    #MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    #MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Literature Name']=='HS 1216+5032'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

print(tmatches_1)
# here we're mixing and matching between Hagen+others and Hennawi+. Deferring mostly to the previous work/GAIA coords
# Note, Hennawi+ used opposite A/B designations relative to the A/B designations used by Hagen+
# We keep using Hagen's as the primary names but we also list Hennawi's
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = 'Dual AGN' # this was classified as a  binary quasar
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = indiv_4.at[j, 'Confirmation Method'] + ' / ' + MAC.at[i, 'Confirmation Method'] #indiv_4.at[j, 'Confirmation Method'] + 
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name2']
    MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    #MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Literature Name']=='J2142-4420'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# here we're going to pretty much just defer to Lusso because there was so little information from Tytler+
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] += ' ; ' + indiv_4.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] +=  ' ; ' +indiv_4.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] +=  ' ; ' + indiv_4.at[j, 'DOI(s)']
    MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] += ' / ' +indiv_4.at[j, 'Selection Method'] 
    MAC.at[i, 'Confirmation Method'] += ' / ' +indiv_4.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    #MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Literature Name']=='PG 1302-2102'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) # 

# the overlap here is between Graham's original paper on the PG target (and all the follow-up) and Graham's own other 2015 paper
# this means we can largely defer to the original Nature paper by Graham
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    #MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] 
    #MAC.at[i, 'Confirmation Method'] += indiv_4.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    #MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Literature Name']=='OQ 208 / Mrk 668'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) #

# mixing and matching here
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    #MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] 
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = indiv_4.at[j, 'Confirmation Method'] + ' / ' + MAC.at[i, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    # I'm going to keep using the coordinates and redshifts from the MAC for now
    #MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    #MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    #MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    #MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    #MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    #MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    #MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    #MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    #MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    #MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    #MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    #MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    #MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    #MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep']
    #MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    #MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Literature Name']=='1635+267'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) #

# here we're deferring to the large number of works (Djorgovski + others) prior to Eftek+2017
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = 'Dual AGN' # Djorgovski referred to this as a binary quasar #indiv_4.at[j, 'System Type'] 
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = indiv_4.at[j, 'Confirmation Method'] + ' / ' + MAC.at[i, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    # The components are mixed when comparing the original work by Djorgovski and the later work by Eftek+2017
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name2']
    MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2'] + ' / ' + MAC.at[i, 'Name1']
    # I'm going to keep using the coordinates and redshifts from the individual for now
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    # we're going to defer to the individual table for coordinates and brightnesses
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep'] # we're taking the separation quoted prior to Eftek+2017
    MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Literature Name']=='2345+007'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) #

# we're deferring largely to the work done prior to Hennawi+
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] + ' / ' + MAC.at[i, 'System Type']
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = "-99 / " + MAC.at[i, 'Confirmation Method'] # indiv_4.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2'] + ' / ' + MAC.at[i, 'Name2']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep'] 
    #MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Literature Name']=='J1536+0441'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) #

# we're going to defer basically completely to the original works focusing on this object
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] + ' / ' + MAC.at[i, 'System Type']
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method']
    MAC.at[i, 'Confirmation Method'] = '-99 / ' + MAC.at[i, 'Confirmation Method'] # indiv_4.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = MAC.at[i, 'Name1'] + ' / ' + indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2'] + ' / ' + MAC.at[i, 'Name2']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2']
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep'] 
    #MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))



indiv_4 = indiv[((indiv['Literature Name']=='Mrk 266'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) #

# we're going to defer basically completely to the original works focusing on this object
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type']
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method'] #indiv_4.at[j, 'Selection Method']
    # come back and fix NaN in selection method
    MAC.at[i, 'Confirmation Method'] = indiv_4.at[j,'Confirmation Method'] # indiv_4.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2'] # 0.028566 on simbad
    # check the redshifts for the two nuclei
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep'] 
    #MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Literature Name']=='SDSS J0927+2943'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) #


# we're going to defer basically completely to the original works focusing on this object
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] + ' / ' + MAC.at[i, 'System Type']
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method'] 
    MAC.at[i, 'Confirmation Method'] = '-99' #indiv_4.at[j,'Confirmation Method'] # indiv_4.at[j, 'Confirmation Method']
    # need to go in later and fix this....
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1']
    MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2'] # 0.028566 on simbad
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep'] 
    #MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


indiv_4 = indiv[((indiv['Literature Name']=='3C 227'))]

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv_4,MAC,60)
print(len(idx1),len(idx1_2)) #

# we're going to defer basically completely to the original works focusing on this object
for i, j in zip(idx2, idx1):
    MAC.at[i, 'Paper(s)'] = indiv_4.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = indiv_4.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = indiv_4.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = indiv_4.at[j, 'System Type'] + ' / ' + MAC.at[i, 'System Type']
    MAC.at[i, 'Notes'] = indiv_4.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] = indiv_4.at[j, 'Selection Method'] + ' / ' + MAC.at[i, 'Selection Method'] 
    MAC.at[i, 'Confirmation Method'] = '-99' #indiv_4.at[j,'Confirmation Method'] # indiv_4.at[j, 'Confirmation Method']
    MAC.at[i, 'Literature Name'] = indiv_4.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = indiv_4.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'Name2'] = indiv_4.at[j, 'Name2']
    MAC.at[i, 'z1'] = indiv_4.at[j, 'z1']  
    MAC.at[i, 'z2'] = indiv_4.at[j, 'z2'] # 0.028566 on simbad
    MAC.at[i, 'z1_type'] = indiv_4.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = indiv_4.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = indiv_4.at[j, 'RA1']
    MAC.at[i, 'RA2'] = indiv_4.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = indiv_4.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = indiv_4.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = indiv_4.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = indiv_4.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = indiv_4.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = indiv_4.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = indiv_4.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = indiv_4.at[j, 'Coordinate_Source2']
    #MAC.at[i, 'Sep'] = indiv_4.at[j, 'Sep'] 
    #MAC.at[i, 'Sep(kpc)'] = indiv_4.at[j, 'Sep(kpc)']
    #MAC.at[i, 'Brightness1'] = indiv_4.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = indiv_4.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = indiv_4.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = indiv_4.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = indiv_4.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = indiv_4.at[j, 'Brightness_type2']

indiv_4.drop(idx1, axis=0, inplace=True)
#indiv_4.drop(idx1_2, axis=0, inplace=True)
indiv_4.reset_index(drop=True, inplace=True)

print(len(indiv_4))


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1
   index           System Type Literature Name       Name1  \
0     97  Binary AGN Candidate        4C+22.25    4C+22.25   
1      2  Binary AGN Candidate             NaN  J1000+2233   

                                    Selection Method Confirmation Method  \
0  Optical Spectroscopy / Optical Fiber Spectrosc...                 -99   
1  Optical Spectroscopy / Fiber Optical Spectrosc...                 -99   

       z1 z1_type                  RA1                 Dec1 Equinox1  \
0  0.4183    spec  10:00:21.8025145008  +22:33:18.641137932    J2000   
1  0.4190    spec          10:00:21.80          +22:33:18.6    J2000   

  Coordinate_waveband1 Coordinate_Source1  Brightness1 Brightness_band1  \
0              Optical           GAIA DR3          NaN              NaN   
1              Optical               SDSS       -100.0             -100   

  Brightness_type1 Name2    z2 z2_type          RA2         Dec2 Equinox2  \
0              NaN   -99 -99.0     -99          -99         

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1322426208.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1
   index    System Type Literature Name            Name1  \
0     19  Binary Quasar    HS 1216+5032    HS 1216+5032A   
1    220  Binary Quasar             NaN  SDSSJ1218+5015A   

                                    Selection Method  \
0  Serendipitous / Optical Imaging / Optical Spec...   
1          Fiber Spectroscopy / Optical Spectroscopy   

                                 Confirmation Method     z1 z1_type  \
0  Optical Imaging / Optical Spectroscopy / Optic...  1.455    spec   
1                          Slit Optical Spectroscopy  1.456    spec   

                   RA1                 Dec1 Equinox1 Coordinate_waveband1  \
0  12:18:41.0133977187  +50:15:35.870275829    J2000              Optical   
1          12:18:40.49          +50:15:43.8    J2000              Optical   

  Coordinate_Source1 Brightness1     Brightness_band1 Brightness_type1  \
0               Gaia       17.09                    V              mag   
1               SDSS        18.8  SDSS g PSF ext-cor

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1322426208.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


...

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1322426208.py:450: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1322426208.py:499: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/2017534566.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['Table_flag'] = 'Table1'


1 1
0


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_47064/1322426208.py:546: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_4.drop(idx1, axis=0, inplace=True)


In [68]:
#MAC

In [69]:
# now removing these from the individual table

#'Literature Name'=='4C+22.25'
#'Literature Name'=='HS 1216+5032A' 
#'Literature Name'=='J2142-4420'
#'Literature Name'=='PG 1302-2102'
#'Literature Name'=='OQ 208 / Mrk 668'
#'Literature Name'=='1635+267'
#'Literature Name'=='2345+007'
#'Literature Name'=='3C 227'
#'Literature Name'=='J1536+0441'
#'Literature Name'=='Mrk 266'
#'Literature Name'=='SDSS J0927+2943'

indiv = indiv[~((indiv['Literature Name']=='4C+22.25')) & ~((indiv['Literature Name']=='HS 1216+5032')) &  
              ~((indiv['Literature Name']=='J2142-4420')) & ~((indiv['Literature Name']=='PG 1302-2102')) & \
              ~((indiv['Literature Name']=='OQ 208 / Mrk 668')) & ~((indiv['Literature Name']=='1635+267')) & \
              ~((indiv['Literature Name']=='2345+007')) & ~((indiv['Literature Name']=='3C 227')) & \
              ~((indiv['Literature Name']=='J1536+0441')) & ~((indiv['Literature Name']=='Mrk 266')) & \
             ~((indiv['Literature Name']=='SDSS J0927+2943'))]

print(len(indiv))



248


In [70]:
# Now to combine all of the tables back together

# as of 4 December 2023, this leaves 247 objects in the individual table

# combining now and then breaking the MAC back apart...
MAC = pd.concat([MAC,indiv_x,indiv_1,indiv,MACnora2])
MAC.reset_index(drop=True, inplace=True)




In [71]:
print(len(MAC))

5936


In [72]:
cols_to_drop = ['level_0','index','SDSS1','rmag1','SDSS2','rmag2','Table_flag',\
                'z','Equinox','Coordinate_waveband','Coordinate_Source','RA','Dec','Name','Vmed','r','delta_z']
#'RA1_deg','Dec1_deg','RA2_deg','Dec2_deg',
for i in cols_to_drop:
    try:
        MAC.drop(labels=str(i), axis=1, inplace=True)
    except:
        print('This column is not available in this version. Must have modified another notebook.')



This column is not available in this version. Must have modified another notebook.


In [73]:
# now we are going to match and add in the 'lost' ojbects from the Inada2010 and Inada2012 tables

# first loading in and matching the Inada tables together and combining them into a single table

inada2010t3 = pd.read_csv('inada2010t3_forlatermatching.csv', sep=',')
inada2012t4 = pd.read_csv('inada2012t4_forlatermatching.csv', sep=',')


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(inada2010t3,inada2012t4,100)

print(len(idx1),len(idx1_2)) # there are no matches even out to 100'' for either AGN 1 to 1 and AGN 2 to 2

# now combining these two tables
inadalost = pd.concat([inada2010t3,inada2012t4])
inadalost.reset_index(drop=True, inplace=True)

coordconvert = SkyCoord(ra = inadalost['RA1'], dec = inadalost['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
inadalost['RA1_deg'] = coordconvert.ra.degree
inadalost['Dec1_deg'] = coordconvert.dec.degree


0 0


In [74]:
#inadalost

In [75]:
# now splitting the MAC again
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']


In [76]:
#MAC.columns

In [77]:
# and now matching against the MAC main table

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(MAC,inadalost,100)

print(len(idx1),len(idx1_2)) # there are no matches even out to 100'' for either AGN 1 to 1 and AGN 2 to 2



# here we're going to defer to Inada because they have separations and redshifts and Yang does not, but \
# we'll also mixandmatch
# AGN 1 is the X-shaped radio source, or at least seems to be the closest match
# so we'll grab coordinates for AGN 2 from Inada
for i, j in zip(idx1, idx2):
    MAC.at[i, 'Paper(s)'] = inadalost.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = inadalost.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = inadalost.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = inadalost.at[j, 'System Type'] + ' / ' + MAC.at[i, 'System Type']
    #MAC.at[i, 'Notes'] = inadalost.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']+ '. AGN 1 is an X-shaped radio source.'
    MAC.at[i, 'Selection Method'] = str(inadalost.at[j, 'Selection Method']) + ' / ' + MAC.at[i, 'Selection Method'] 
    MAC.at[i, 'Confirmation Method'] = str(inadalost.at[j,'Confirmation Method']) + ' / ' + MAC.at[i, 'Confirmation Method']
    #MAC.at[i, 'Literature Name'] = inadalost.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = inadalost.at[j, 'Name1'] + ' / ' + MAC.at[i, 'Name1']
    MAC.at[i, 'Name2'] = inadalost.at[j, 'Name2']
    MAC.at[i, 'z1'] = inadalost.at[j, 'z1']  
    MAC.at[i, 'z2'] = inadalost.at[j, 'z2'] # 0.028566 on simbad
    MAC.at[i, 'z1_type'] = inadalost.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = inadalost.at[j, 'z2_type']
    #MAC.at[i, 'RA1'] = inadalost.at[j, 'RA1']
    MAC.at[i, 'RA2'] = inadalost.at[j, 'RA2']
    #MAC.at[i, 'Dec1'] = inadalost.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = inadalost.at[j, 'Dec2']
    #MAC.at[i, 'Equinox1'] = inadalost.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = inadalost.at[j, 'Equinox2']
    #MAC.at[i, 'Coordinate_waveband1'] = inadalost.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = inadalost.at[j, 'Coordinate_waveband2']
    #MAC.at[i, 'Coordinate_Source1'] = inadalost.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = inadalost.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = inadalost.at[j, 'Sep'] 
    #MAC.at[i, 'Sep(kpc)'] = inadalost.at[j, 'Sep(kpc)']
    #MAC.at[i, 'Brightness1'] = inadalost.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = inadalost.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = inadalost.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = inadalost.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = inadalost.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = inadalost.at[j, 'Brightness_type2']

inadalost.drop(idx2, axis=0, inplace=True)
##inadalost.drop(idx1_2, axis=0, inplace=True)
inadalost.reset_index(drop=True, inplace=True)



tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(MACnora2,inadalost,10)

print(len(idx1),len(idx1_2)) # there are no matches even out to 100'' for either AGN 1 to 1 and AGN 2 to 2


# for the Rusu target, we'll going to pretty much defer exclusively to Inada
# if you match out to 50'' or so, you get an improper match with a Liu+2014, but these are not associated with one another

for i, j in zip(idx1, idx2):
    MAC.at[i, 'Paper(s)'] = inadalost.at[j, 'Paper(s)'] + ' ; ' + MAC.at[i, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] = inadalost.at[j, 'BibCode(s)'] + ' ; ' + MAC.at[i, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] = inadalost.at[j, 'DOI(s)'] + ' ; ' + MAC.at[i, 'DOI(s)']
    MAC.at[i, 'System Type'] = inadalost.at[j, 'System Type'] + ' / ' + MAC.at[i, 'System Type']
    #MAC.at[i, 'Notes'] = inadalost.at[j, 'Notes'] + ' ' + MAC.at[i, 'Notes']
    MAC.at[i, 'Selection Method'] = str(inadalost.at[j, 'Selection Method']) + ' / ' + MAC.at[i, 'Selection Method'] 
    MAC.at[i, 'Confirmation Method'] = str(inadalost.at[j,'Confirmation Method']) + ' / ' + MAC.at[i, 'Confirmation Method']
    #MAC.at[i, 'Literature Name'] = inadalost.at[j, 'Literature Name']
    MAC.at[i, 'Name1'] = inadalost.at[j, 'Name1']
    MAC.at[i, 'Name2'] = inadalost.at[j, 'Name2']
    MAC.at[i, 'z1'] = inadalost.at[j, 'z1']  
    MAC.at[i, 'z2'] = inadalost.at[j, 'z2'] # 0.028566 on simbad
    MAC.at[i, 'z1_type'] = inadalost.at[j, 'z1_type']
    MAC.at[i, 'z2_type'] = inadalost.at[j, 'z2_type']
    MAC.at[i, 'RA1'] = inadalost.at[j, 'RA1']
    MAC.at[i, 'RA2'] = inadalost.at[j, 'RA2']
    MAC.at[i, 'Dec1'] = inadalost.at[j, 'Dec1']
    MAC.at[i, 'Dec2'] = inadalost.at[j, 'Dec2']
    MAC.at[i, 'Equinox1'] = inadalost.at[j, 'Equinox1']
    MAC.at[i, 'Equinox2'] = inadalost.at[j, 'Equinox2']
    MAC.at[i, 'Coordinate_waveband1'] = inadalost.at[j, 'Coordinate_waveband1']
    MAC.at[i, 'Coordinate_waveband2'] = inadalost.at[j, 'Coordinate_waveband2']
    MAC.at[i, 'Coordinate_Source1'] = inadalost.at[j, 'Coordinate_Source1']
    MAC.at[i, 'Coordinate_Source2'] = inadalost.at[j, 'Coordinate_Source2']
    MAC.at[i, 'Sep'] = inadalost.at[j, 'Sep'] 
    #MAC.at[i, 'Sep(kpc)'] = inadalost.at[j, 'Sep(kpc)']
    #MAC.at[i, 'Brightness1'] = inadalost.at[j, 'Brightness1']
    #MAC.at[i, 'Brightness_band1'] = inadalost.at[j, 'Brightness_band1']
    #MAC.at[i, 'Brightness_type1'] = inadalost.at[j, 'Brightness_type1']
    #MAC.at[i, 'Brightness2'] = inadalost.at[j, 'Brightness2']
    #MAC.at[i, 'Brightness_band2'] = inadalost.at[j, 'Brightness_band2']
    #MAC.at[i, 'Brightness_type2'] = inadalost.at[j, 'Brightness_type2']

inadalost.drop(idx2, axis=0, inplace=True)
#inadalost.drop(idx1_2, axis=0, inplace=True)
inadalost.reset_index(drop=True, inplace=True)



1 1
1 1


In [78]:
# now concatenating all tables back together
MAC = pd.concat([MAC,MACnora2,inadalost])
MAC.reset_index(drop=True, inplace=True)

#MAC.columns

cols_to_drop = ['level_0','index','Table_flag','i_band_psfmag(extcorr)1', 'i_band_psfmag(extcorr)2', 'Sep_as',\
                'z','Equinox','Coordinate_waveband','Coordinate_Source','RA','Dec','Name','Vmed','r','delta_z',\
               'Comment', 'imag1', 'imag2', 'theta', 'Com']
#'RA1_deg','Dec1_deg','RA2_deg','Dec2_deg',
for i in cols_to_drop:
    try:
        MAC.drop(labels=str(i), axis=1, inplace=True)
    except:
        print('This column is not available in this version. Must have modified another notebook.')


This column is not available in this version. Must have modified another notebook.
This column is not available in this version. Must have modified another notebook.
This column is not available in this version. Must have modified another notebook.
This column is not available in this version. Must have modified another notebook.
This column is not available in this version. Must have modified another notebook.
This column is not available in this version. Must have modified another notebook.
This column is not available in this version. Must have modified another notebook.
This column is not available in this version. Must have modified another notebook.
This column is not available in this version. Must have modified another notebook.
This column is not available in this version. Must have modified another notebook.


In [79]:
# and now we're adding in the missing rows from the Ju+2013 paper

ju2013lost = pd.read_csv('Ju2013losttargets.csv', sep=',')
ju2013lost.reset_index(drop=True, inplace=True)

coordconvert = SkyCoord(ra = ju2013lost['RA1'], dec = ju2013lost['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
ju2013lost['RA1_deg'] = coordconvert.ra.degree
ju2013lost['Dec1_deg'] = coordconvert.dec.degree

ju2013lost['RA2_deg'] = coordconvert.ra.degree
ju2013lost['Dec2_deg'] = coordconvert.dec.degree

In [80]:
# now splitting the MAC again
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']


In [81]:
# and now matching against the MAC main table


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(MAC,ju2013lost,100)

print(len(idx1),len(idx1_2)) # there are no matches even out to 100'' for either AGN 1 to 1 and AGN 2 to 2


for i, j in zip(idx1, idx2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + ju2013lost.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + ju2013lost.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + ju2013lost.at[j, 'DOI(s)']
    MAC.at[i, 'System Type'] += ' / ' + ju2013lost.at[j, 'System Type']
    MAC.at[i, 'Notes'] += ju2013lost.at[j, 'Notes'] + ' ' + 'AGN 1 is selected a binary candidate by Ju+2013.'
    MAC.at[i, 'Selection Method'] = str(ju2013lost.at[j, 'Selection Method']) + ' / ' + MAC.at[i, 'Selection Method'] 
#    MAC.at[i, 'Confirmation Method'] = str(ju2013lost.at[j,'Confirmation Method']) + ' / ' + MAC.at[i, 'Confirmation Method']
#    MAC.at[i, 'Literature Name'] = ju2013lost.at[j, 'Literature Name']


ju2013lost.drop(idx2, axis=0, inplace=True)
###ju2013lost.drop(idx1_2, axis=0, inplace=True)
ju2013lost.reset_index(drop=True, inplace=True)


#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(MACnora2,ju2013lost,100)

#print(len(idx1),len(idx1_2)) # there are no matches even out to 100'' for either AGN 1 to 1 and AGN 2 to 2



1 1


In [82]:
#tmatches_1

In [83]:
## now concatenating all tables back together
MAC = pd.concat([MAC,MACnora2,ju2013lost])
MAC.reset_index(drop=True, inplace=True)
#
##MAC.columns
#
#cols_to_drop = ['level_0','index','Table_flag','i_band_psfmag(extcorr)1', 'i_band_psfmag(extcorr)2', 'Sep_as',\
#                'z','Equinox','Coordinate_waveband','Coordinate_Source','RA','Dec','Name','Vmed','r','delta_z',\
#               'Comment', 'imag1', 'imag2', 'theta', 'Com']
##'RA1_deg','Dec1_deg','RA2_deg','Dec2_deg',
#for i in cols_to_drop:
#    try:
#        MAC.drop(labels=str(i), axis=1, inplace=True)
#    except:
#        print('This column is not available in this version. Must have modified another notebook.')



In [84]:
comerford2009lost = pd.read_csv('Comerford2009_offsettargets.csv', sep=',')
comerford2009lost.reset_index(drop=True, inplace=True)

coordconvert = SkyCoord(ra = comerford2009lost['RA1'], dec = comerford2009lost['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
comerford2009lost['RA1_deg'] = coordconvert.ra.degree
comerford2009lost['Dec1_deg'] = coordconvert.dec.degree

comerford2009lost['RA2_deg'] = coordconvert.ra.degree
comerford2009lost['Dec2_deg'] = coordconvert.dec.degree

In [85]:
# now splitting the MAC again
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']


In [86]:
## and now matching against the MAC main table


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(MAC,comerford2009lost,100)

print(len(idx1),len(idx1_2)) # there are no matches even out to 100'' for either AGN 1 to 1 and AGN 2 to 2


tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(MACnora2,comerford2009lost,50)

print(len(idx1),len(idx1_2)) # there are no matches even out 50'' for either AGN 1 to 1 and AGN 2 to 2



0 0
0 0


In [87]:
MAC = pd.concat([MAC,MACnora2,comerford2009lost])
MAC.reset_index(drop=True, inplace=True)

In [88]:
# now loading in the Barrows+ tables of offset AGNs

barrows2016lost = pd.read_csv('barrows2016_offsettargets.csv', sep=',')
barrows2016lost.reset_index(drop=True, inplace=True)

coordconvert = SkyCoord(ra = barrows2016lost['RA1'], dec = barrows2016lost['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
barrows2016lost['RA1_deg'] = coordconvert.ra.degree
barrows2016lost['Dec1_deg'] = coordconvert.dec.degree

barrows2016lost['RA2_deg'] = coordconvert.ra.degree
barrows2016lost['Dec2_deg'] = coordconvert.dec.degree


In [89]:
# now splitting the MAC again
MACnora2 = MAC[MAC['RA2']=='-99']
MAC = MAC[MAC['RA2']!='-99']


In [90]:
tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(MAC,barrows2016lost,100)

print(len(idx1),len(idx1_2)) 


# the match here is with Liu+2011; if Barrows+ is selecting their tagrets as dual SMBH candidates, and the other Liu \
# target is at 72'' ish, then this could* constitute a triple SMBH candidate... but we're not going too refer \
# to it as that yet...

for i, j in zip(idx1, idx2):
    MAC.at[i, 'Paper(s)'] += ' ; ' + barrows2016lost.at[j, 'Paper(s)']
    MAC.at[i, 'BibCode(s)'] += ' ; ' + barrows2016lost.at[j, 'BibCode(s)']
    MAC.at[i, 'DOI(s)'] += ' ; ' + barrows2016lost.at[j, 'DOI(s)']
    MAC.at[i, 'System Type'] += ' / ' + barrows2016lost.at[j, 'System Type']
    MAC.at[i, 'Notes'] += 'AGN 2 is selected a dual SMBH candidate by Barrows+.'
    MAC.at[i, 'Selection Method'] = str(barrows2016lost.at[j, 'Selection Method']) + ' / ' + MAC.at[i, 'Selection Method'] 
#    MAC.at[i, 'Confirmation Method'] = str(barrows2016lost.at[j,'Confirmation Method']) + ' / ' + MAC.at[i, 'Confirmation Method']
#    MAC.at[i, 'Literature Name'] = barrows2016lost.at[j, 'Literature Name']


barrows2016lost.drop(idx2, axis=0, inplace=True)
###barrows2016lost.drop(idx1_2, axis=0, inplace=True)
barrows2016lost.reset_index(drop=True, inplace=True)


#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(MACnora2,barrows2016lost,50)

#print(len(idx1),len(idx1_2)) # there are no matches even out 50'' for either AGN 1 to 1 and AGN 2 to 2

#tmatches_1

1 1


In [91]:
MAC = pd.concat([MAC,MACnora2,barrows2016lost])
MAC.reset_index(drop=True, inplace=True)

In [92]:
#MAC

In [93]:
# and clipping out the columns we don't need...

MAC = MAC[['System Type','Literature Name','Selection Method','Confirmation Method','Name1','z1','z1_type','RA1',\
           'Dec1','Coordinate_waveband1','Coordinate_Source1','Equinox1','Brightness1','Brightness_band1',\
           'Brightness_type1','Name2','z2','z2_type','RA2','Dec2','Equinox2','Coordinate_waveband2',\
           'Coordinate_Source2','Brightness2','Brightness_band2','Brightness_type2','dV','Sep','Sep(kpc)',\
           'dV_rwp','Paper(s)','BibCode(s)','DOI(s)','Notes']]

MAC.reset_index(drop=True, inplace=True)


In [94]:
MAC.to_csv('MAC_DR0p5_16Jan2024.csv', sep=',')



In [95]:
## matching against the MAC table with RA2 vals
#tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib_doubles(indiv,MAC,60)
#
#print(len(idx1),len(idx1_2)) # 
#
## now matching just against the MAC table without an RA2 value
#tunique, tmatches, idx1, idx2 = match_tables_fib_singles(indiv,MACnora2,60)
#
#print(len(idx1)) #,len(idx)) # 






In [96]:
# DEPRECATED BELOW HERE!

In [97]:
# just for testing purposes here to make the beta version of the full list (no matching of individual lines to \
# the larger table)


# MAC = pd.concat([MAC,MACnora2,indiv,indivnora])


In [98]:
# MAC.drop(labels=['z','Equinox','Coordinate_waveband','Coordinate_Source','RA','Dec','Name','Vmed',\
#                  'delta_z','Literature Name','level_0','index','SDSS1','SDSS2','dV_rwp'], axis=1, inplace=True)


In [99]:
# MAC.drop(labels=['Table_flag','r'], axis=1, inplace=True)

In [100]:
# MAC

In [101]:
# list(MAC.columns.values)

In [102]:
# MAC = MAC[['System Type','Selection Method','Confirmation Method','Name1','z1','z1_type','RA1','Dec1','RA1_deg',\
#            'Dec1_deg','Equinox1','Coordinate_waveband1','Coordinate_Source1','rmag1','Brightness1',\
#            'Brightness_band1','Brightness_type1','Name2','z2','z2_type','RA2','Dec2','RA2_deg','Dec2_deg',\
#            'Equinox2','rmag2','Coordinate_waveband2','Coordinate_Source2','Brightness2','Brightness_band2',\
#            'Brightness_type2','dV','Sep','Sep(kpc)','Paper(s)','BibCode(s)','DOI(s)','Notes']]



In [103]:
# MAC

In [104]:
# MAC.to_csv('MAC_DR1_beta.csv', sep=',')